In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib_inline.backend_inline import set_matplotlib_formats
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import calinski_harabasz_score
from typing import Dict, Optional, Tuple, Any, List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.metrics import precision_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from scipy import stats
from collections import defaultdict
from math import log2
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr
import re, gc
from typing import Optional, Iterable
from time import perf_counter
from datetime import datetime, timedelta
from sklearn.metrics import ndcg_score
import warnings, math

def _rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def _spearman(y_true, y_pred):
    c = spearmanr(y_true, y_pred).correlation
    return float(c) if np.isfinite(c) else np.nan

import warnings
warnings.filterwarnings('ignore')



In [3]:
# 한글 폰트
plt.rc("font", family = 'AppleGothic')
plt.rc("axes", unicode_minus = False)

# 글씨 선명하게
set_matplotlib_formats("retina")

In [4]:
ads_sch = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/스케줄.csv')
print(f"데이터 수 : {ads_sch.shape}")
ads_sch.head()

데이터 수 : (527696, 11)


,sch_idx,ads_idx,sch_type,mda_idx_arr,sch_sdatetime,sch_edatetime,sch_adv_pay,sch_ads_pay,sch_mda_pay,sch_clk_num,sch_turn_num
0,1257478,463989,M,"562,563",2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,14,99999999,14
1,1257477,463989,A,NaN,2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,12,99999999,14
2,1257476,463988,M,"562,563",2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,14,99999999,18
3,1257475,463988,A,NaN,2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,12,99999999,18
4,1257474,463987,M,"562,563",2025-09-11 09:41:55,9999-12-31 23:59:59,18,18,14,99999999,7


In [5]:
ads_sch.columns

Index(['sch_idx', 'ads_idx', 'sch_type', 'mda_idx_arr', 'sch_sdatetime',
       'sch_edatetime', 'sch_adv_pay', 'sch_ads_pay', 'sch_mda_pay',
       'sch_clk_num', 'sch_turn_num'],
      dtype='object')

In [6]:
ads_time = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_시간별적립보고서(최종).csv')
print(f"데이터 수 : {ads_time.shape}")
ads_time.head()

데이터 수 : (279916, 10)


,rpt_time_date,rpt_time_time,ads_idx,mda_idx,rpt_time_clk,rpt_time_turn,rpt_time_acost,rpt_time_earn,weekday,week_type
0,2025-07-27,0,11059,398,5,0,0,0,6,주말
1,2025-07-27,0,13209,583,3,0,0,0,6,주말
2,2025-07-27,0,13928,303,3,3,510,360,6,주말
3,2025-07-27,0,13928,324,3,3,510,360,6,주말
4,2025-07-27,0,13928,340,6,6,1020,720,6,주말


In [7]:
ads_time.columns

Index(['rpt_time_date', 'rpt_time_time', 'ads_idx', 'mda_idx', 'rpt_time_clk',
       'rpt_time_turn', 'rpt_time_acost', 'rpt_time_earn', 'weekday',
       'week_type'],
      dtype='object')

In [8]:
ads_domain = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/광고도메인리스트_1년치.csv')
print(f"데이터 수 : {ads_domain.shape}")
ads_domain.head()

데이터 수 : (233408, 24)


,Unnamed: 0,domain,row_id,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,...,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,0,금융,153088,154946,KoM69LPFUB,8,50,3,7,어센틱금융그룹 내보험조회,...,2025-02-12 10:00:00,2029-11-29 00:00:00,0,100,7,800,640,2444400,NONE,2025-02-12 10:18:46
1,21,게임,135518,137375,RBjgpLVOmR,71,910,3,8,DK모바일 리본 사전예약,...,2025-01-10 18:00:00,2030-01-01 00:00:00,0,100,7,200,150,2431500,NONE,2025-01-10 18:48:52
2,23,게임,427141,429013,TUSVjcBYtK,76,1355,3,5,인생존망겜(레벨 67 달성),...,2025-07-11 16:23:32,9999-12-31 23:59:59,0,100,2,9600,7200,2460500,NONE,2025-07-11 16:23:32
3,31,게임,161321,163182,OLNpNxxGJm,76,1355,3,5,고블린 우드: 타이쿤 유휴 심,...,2025-02-27 16:00:00,9999-12-31 23:59:59,0,100,2,4480,3360,2451800,NONE,2025-02-26 17:05:49
4,34,게임,161392,163254,r78MSmvuQk,76,1355,3,5,Save the Pets,...,2025-02-27 16:00:00,2031-01-03 00:00:00,0,100,1,1440,1080,2450800,NONE,2025-02-26 17:57:33


In [9]:
ads_domain.columns

Index(['Unnamed: 0', 'domain', 'row_id', 'ads_idx', 'ads_code', 'aff_idx',
       'adv_idx', 'ads_type', 'ads_category', 'ads_name', 'ads_icon_img',
       'ads_summary', 'ads_save_way', 'ads_day_cap', 'ads_sdate', 'ads_edate',
       'ads_age_min', 'ads_age_max', 'ads_os_type', 'ads_contract_price',
       'ads_reward_price', 'ads_order', 'ads_rejoin_type', 'regdate'],
      dtype='object')

In [10]:
ads_list = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고목록.csv')
print(f"데이터 수 : {ads_list.shape}")
ads_list.head()

데이터 수 : (445039, 21)


,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,...,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,160,tr71opm6C1,6,10,1,1,리니지레드나이츠,https://lh3.googleusercontent.com/oDGXxngO9oOP...,[앱설명]<br />[설치형] 매력만점 영웅들과 소환수들이 함께하는 취향저격 RPG...,받기,...,2016-12-01 00:00:00,9999-12-31 23:59:59,0,100,2,170,150,982800,NONE,2017-04-06 13:19:53
1,284,xlNpfkgtLX,6,10,1,1,강철의함대:Ocean Overlord,http://nextapps-nas.aws.appang.kr/icon/2016122...,"[앱설명]<br />[설치형] 한, 중, 일 아시아 유저들과 함께 대규모 해상 전투...",받기,...,2016-12-29 00:00:00,9999-12-31 23:59:59,0,100,2,180,160,987200,NONE,2017-01-10 10:37:22
2,292,7Hj8V5QKiy,6,10,1,1,스노우 SNOW,http://nextapps-nas.aws.appang.kr/icon/2016092...,"[앱설명]<br />[설치형] 셀카, 얼굴인식 스티커, 꿀잼 카메라<br /><br...",받기,...,2016-12-29 00:00:00,9999-12-31 23:59:59,0,100,2,160,160,987300,NONE,2017-01-02 12:51:34
3,304,1RfxHJcuEe,6,10,2,1,서머너즈 워: 천공의 아레나,http://nextapps-nas.aws.appang.kr/icon/EecpHzP...,[앱설명]<br />[실행형] 전 세계 5000만 소환사들을 사로잡은 명작 RPG<...,최초 오픈,...,2017-01-01 00:00:00,9999-12-31 23:59:59,0,100,2,230,210,931000,NONE,2017-03-30 17:05:23
4,306,l3GyilYEVk,6,10,1,1,하이마트,http://nextapps-nas.aws.appang.kr/icon/2016032...,새롭게 단장한 하이마트 쇼핑몰 앱!,받기,...,2017-01-01 00:00:00,2019-11-01 00:00:00,0,100,2,170,140,1155100,NONE,2017-06-12 15:24:46


In [11]:
ads_list.columns

Index(['ads_idx', 'ads_code', 'aff_idx', 'adv_idx', 'ads_type', 'ads_category',
       'ads_name', 'ads_icon_img', 'ads_summary', 'ads_save_way',
       'ads_day_cap', 'ads_sdate', 'ads_edate', 'ads_age_min', 'ads_age_max',
       'ads_os_type', 'ads_contract_price', 'ads_reward_price', 'ads_order',
       'ads_rejoin_type', 'regdate'],
      dtype='object')

In [12]:
ads_rwd = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_광고적립.csv')
print(f"데이터 수 : {ads_rwd.shape}")
ads_rwd.head()

데이터 수 : (1475031, 14)


,rwd_idx,ads_idx,ads_code,mda_idx,pub_sub_rel_id,dvc_idx,advid,click_key,adv_cost,earn_cost,click_date,regdate,ctit,cost_diff
0,238304818,413490,CFZ6lzEmeD,22,27496512,32947806,09b6a808-4f76-4584-87c5-626f21f472a0,b0e22dba943810794d25c993dff425c4e16d0744,190,120,2025-07-26 00:00:31,2025-07-26 00:05:02,271,70
1,238304829,413490,CFZ6lzEmeD,22,27496512,47387244,ba4fd0c8-4f06-4dfa-9c3f-c6b39ee7066e,0679ff4c6a14beba296e44d7ffdd88129f42ffbf,190,120,2025-07-26 00:00:32,2025-07-26 00:08:17,465,70
2,238304816,413490,CFZ6lzEmeD,22,27496512,32425124,64c675d0-8edd-4478-9906-a806651fdf92,68eb28d636724061da21e490a1afff6a39cd78f7,190,120,2025-07-26 00:00:42,2025-07-26 00:04:43,241,70
3,238304817,413490,CFZ6lzEmeD,22,27496513,34006545,a08ebb04-e70a-4492-89d4-17fa96bf8c75,37da42f12b6f732d8c8bf1a609c01e569b7f8a09,190,120,2025-07-26 00:00:54,2025-07-26 00:04:45,231,70
4,238304802,133653,rl8ud4uNQC,801,1,56867331,dbe54f40-41b5-4423-ae2f-048fd6ecf5af,624b1bbc57c9ea81005c74e1956d48e76d974816,260,195,2025-07-26 00:01:07,2025-07-26 00:02:04,57,65


In [13]:
ads_rwd.columns

Index(['rwd_idx', 'ads_idx', 'ads_code', 'mda_idx', 'pub_sub_rel_id',
       'dvc_idx', 'advid', 'click_key', 'adv_cost', 'earn_cost', 'click_date',
       'regdate', 'ctit', 'cost_diff'],
      dtype='object')

In [14]:
ads_info = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고참여정보.csv')
print(f"데이터 수 : {ads_info.shape}")
ads_info.head()

데이터 수 : (16854865, 14)


,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,click_date,exp_day,network,user_ip,converted
0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,2025-08-17 21:07:37,2025-09-16,0,16.184.28.219,0
1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,2025-07-26 02:18:24,2025-08-25,NaN,35.78.117.76,1
2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,2025-08-12 18:17:59,2025-09-11,WIFI,39.7.55.192,1
3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,2025-08-17 03:35:02,2025-09-16,0,211.252.100.157,0
4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,2025-08-18 08:01:26,2025-09-17,0,3.38.148.211,0


In [15]:
ads_info.columns

Index(['click_key', 'ads_idx', 'dvc_idx', 'mda_idx', 'pub_sub_rel_id',
       'contract_price', 'media_price', 'click_day', 'click_time',
       'click_date', 'exp_day', 'network', 'user_ip', 'converted'],
      dtype='object')

In [16]:
ads_join = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/참여적립조인.csv')
print(f"데이터 수 : {ads_join.shape}")
ads_join.head()

데이터 수 : (16854865, 22)


,Unnamed: 0,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,...,network,user_ip,rwd_idx,ads_code,advid,adv_cost,earn_cost,regdate,ctit,conversion
0,0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,...,0,16.184.28.219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,...,NaN,35.78.117.76,238313226.0,yuRe4YoPuZ,40ceb01c-9b60-4452-bacd-0e061fb18ab4,180.0,170.0,2025-07-26 02:19:21,57.0,1
2,2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,...,WIFI,39.7.55.192,239017177.0,WuIp8HKmt9,73793f44-a471-4351-ae1c-d09accaf223a,170.0,120.0,2025-08-12 18:19:21,82.0,1
3,3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,...,0,211.252.100.157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,...,0,3.38.148.211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [17]:
ads_join.columns

Index(['Unnamed: 0', 'click_key', 'ads_idx', 'dvc_idx', 'mda_idx',
       'pub_sub_rel_id', 'contract_price', 'media_price', 'click_day',
       'click_time', 'click_date', 'exp_day', 'network', 'user_ip', 'rwd_idx',
       'ads_code', 'advid', 'adv_cost', 'earn_cost', 'regdate', 'ctit',
       'conversion'],
      dtype='object')

In [18]:
# ads_total = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/아이브1년치_참여데이터_통합.csv')
# print(f"데이터 수 : {ads_total.shape}")
# ads_total.head()

In [19]:
ads_segment = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/ads_segment.csv')
print(f"데이터 수 : {ads_segment.shape}")
ads_segment.head()

데이터 수 : (9517, 40)


,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,...,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster
0,0,6508,1,1,2,0,10,금융,7,2381500,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
1,1,6985,2,30,46,0,7,금융,7,1201938,...,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0
2,2,8327,1,6,11,0,10,미디어/컨텐츠,7,1673458,...,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0
3,3,9264,1,1,1,0,10,금융,7,2372800,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
4,4,9716,2,4,6,1,8,생활,2,2699900,...,LOW,MEDIUM_LOW,2,0,1,1,0,4,SMALL,0


In [20]:
ads_segment.columns

Index(['Unnamed: 0', 'ads_idx', 'media_count', 'user_count', 'total_clicks',
       'total_conversions', 'ads_category', 'domain', 'ads_os_type',
       'ads_order', 'ctit_mean', 'ctit_median', 'ads_rejoin_type',
       'contract_price', 'media_price', 'first_click', 'last_click',
       'ads_name', 'ads_sdate', 'expire', 'days_active',
       'daily_avg_conversions', 'cvr', 'margin', 'roi', 'total_net_return',
       'ads_level_detailed', 'daily_clicks', 'daily_users', 'scale_level',
       'performance_level', 'ads_level', 'media_score', 'conv_score',
       'clicks_score', 'stability_score', 'cvr_score', 'total_score',
       'ads_size', 'cluster'],
      dtype='object')

In [21]:
media_portfolio = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/media_portfolio.csv')
print(f"데이터 수 : {media_portfolio.shape}")
media_portfolio.head()

데이터 수 : (189, 50)


,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,...,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit
0,0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,7,...,16.666667,12.500000,0.00000,16.666667,0.000000,0.000000,4.166667,0.000000,12.500000,3003668
1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,...,0.000000,13.559322,0.00000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547
2,2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,19,...,0.000000,8.695652,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.347826,182366
3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,...,3.012048,11.445783,0.60241,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770
4,4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,10,...,0.000000,10.526316,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526316,2690


In [22]:
media_portfolio.columns

Index(['Unnamed: 0', 'mda_idx', 'user_count', 'total_clicks',
       'total_conversions', 'first_click', 'last_click', 'days_active',
       'daily_avg_conversions', 'LARGE', 'MEDIUM', 'MEGA', 'SMALL',
       'total_ads', 'MEGA_ratio', 'LARGE_ratio', 'MEDIUM_ratio', 'SMALL_ratio',
       'ads_category_0_pct', 'ads_category_1_pct', 'ads_category_2_pct',
       'ads_category_3_pct', 'ads_category_4_pct', 'ads_category_5_pct',
       'ads_category_6_pct', 'ads_category_7_pct', 'ads_category_8_pct',
       'ads_category_10_pct', 'ads_category_11_pct', 'ads_category_13_pct',
       'ads_os_type_1_pct', 'ads_os_type_2_pct', 'ads_os_type_3_pct',
       'ads_os_type_7_pct', 'domain_게임_pct', 'domain_교육_pct', 'domain_금융_pct',
       'domain_기타_pct', 'domain_미디어/컨텐츠_pct', 'domain_뷰티_pct',
       'domain_비영리/공공_pct', 'domain_생활_pct', 'domain_식당/카페_pct',
       'domain_식음료_pct', 'domain_운동/스포츠_pct', 'domain_운세_pct',
       'domain_의료/건강_pct', 'domain_채용_pct', 'domain_커머스_pct',
       'expected_tota

# ---- 최소 피처 셋(없으면 0 채움; ensure_feature_matrix가 안전 처리) ----
FEAT_MIN = [
    # 매체 전반(OBS 집계)
    "mda_clk_global",          # OBS 전체 클릭 수 (규모/활동성 proxy)
    "mda_conv_global",         # OBS 전체 전환 수
    "mda_cvr_global",          # OBS 전환율 = conv/clk
    "mda_pop_norm",            # 매체 전반 인기(표준화 conv)

    # 카테고리 × 매체 적합도(OBS 집계)
    "cat_cvr",                 # 해당 광고 카테고리에서의 매체 CVR (conv/clk)
    "cat_share",               # 매체 전체 전환 중 해당 카테고리 비중

    # 광고 가격/속성(ads_list)
    "price_reward_log",        # log(ads_reward_price+1)
    "price_contract_log",      # log(ads_contract_price+1)
    "price_ratio",             # reward/contract (NaN→0)

    # media_portfolio(있으면 사용, 없으면 0)
    "mda_size",                # 규모(예: 소/중/대 수치화 or 점수)
    "active_ratio",            # 활성 지표
    "day_ratio", "weekend_ratio"
]

# 모델링

In [3]:
# ==============================================
#  Ad × Media 추천 (Attribute-only, Cold-start Safe)
#  - 과거 "광고×매체" 페어 성과/CF 전부 배제
#  - 허용 피처만: mda_cvr_global, mda_pop_norm, cat_cvr, cat_share,
#                 price_reward_log, price_contract_log, price_ratio,
#                 dom_cvr, dom_share, os_cvr, os_share
#  - (선택) media_portfolio: mda_size, active_ratio, day_ratio, weekend_ratio
#  - 학습/예측 스케일 다운: 광고 대상 제한 + 후보 축소 + 배치 예측
# ==============================================

warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 공통 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str = "ads_category") -> pd.DataFrame:
    """컬럼을 문자열로 통일(merge dtype 오류 방지)"""
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s):
    """Series를 문자열/소문자/trim으로 표준화"""
    return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    """cand 후보 중 존재하는 첫 컬럼명 반환"""
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    """df에서 cands 중 하나를 찾아 out_col에 표준화하여 채움. 없으면 '__na__'."""
    col = _pick_col(df.columns, cands)
    if col is None:
        df[out_col] = "__na__"
    else:
        df[out_col] = _norm_cat(df[col])
    return df

# -------------------------------
# 1) 윈도우 (날짜 직접 지정)
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6  # tgt_end 미지정 시, 최대 7일(0~6) 사용
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")

    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()

    # OBS 끝이 없으면 TGT 시작 전날
    if obs_end is None:
        obs_end = (tgt_start - pd.Timedelta(days=1)).normalize()
    else:
        obs_end = pd.to_datetime(obs_end).normalize()

    # TGT 끝이 없으면 min(데이터 마지막날, tgt_start+6일)
    last_date = t["date"].max().normalize()
    if tgt_end is None:
        tgt_end = min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize())
    else:
        tgt_end = pd.to_datetime(tgt_end).normalize()

    # 필터링
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()

    # 기존 반환 포맷과 호환
    month_start = obs_start
    month_end   = tgt_end
    cutoff      = (tgt_start - pd.Timedelta(days=1)).normalize()
    return obs, tgt, month_start, month_end, cutoff

# -------------------------------
# 2) 속성 컨텍스트(OBS만 사용)
# -------------------------------
def build_attr_context(obs: pd.DataFrame, ads_list: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # 매체 전반 (OBS)
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"),
                      mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0, np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체 (OBS)
    al = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])

    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"] / cat_mda["clk"].replace(0, np.nan)).fillna(0.0)

    # 각 mda에서 카테고리 전환 비중(cat_share) = conv_cat / conv_total_mda
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"] /
                            cat_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)

    # --- NEW: 도메인 / OS 적합도 (OBS 로그에 있을 경우만) ---
    dom_col = _pick_col(t.columns, ["req_domain","domain","site_domain","publisher_domain"])
    os_col  = _pick_col(t.columns, ["os_type","device_os","os"])

    dom_mda = None
    if dom_col is not None:
        t_dom = t.copy()
        t_dom[dom_col] = _norm_cat(t_dom[dom_col])
        dom_mda = (t_dom.groupby(["mda_idx", dom_col], as_index=False)
                        .agg(clk=("clk","sum"), conv=("conv","sum")))
        dom_mda["dom_cvr"] = (dom_mda["conv"] / dom_mda["clk"].replace(0, np.nan)).fillna(0.0)
        mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
        dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
        dom_mda["dom_share"] = (dom_mda["conv"] / dom_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)
        dom_mda = dom_mda.rename(columns={dom_col: "ads_domain"})

    os_mda = None
    if os_col is not None:
        t_os = t.copy()
        t_os[os_col] = _norm_cat(t_os[os_col])
        os_mda = (t_os.groupby(["mda_idx", os_col], as_index=False)
                        .agg(clk=("clk","sum"), conv=("conv","sum")))
        os_mda["os_cvr"] = (os_mda["conv"] / os_mda["clk"].replace(0, np.nan)).fillna(0.0)
        mda_tot3 = os_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
        os_mda = os_mda.merge(mda_tot3, on="mda_idx", how="left")
        os_mda["os_share"] = (os_mda["conv"] / os_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)
        os_mda = os_mda.rename(columns={os_col: "os_type"})

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda, "os_mda": os_mda}

# -------------------------------
# 3) 학습 대상 광고 선별 (TGT에서 양성 존재 + 상한)
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000,
                            rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1, int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성(속성 기반, 강력 축소)
# -------------------------------
def create_candidates(
    ads_list: pd.DataFrame,
    mda_all: pd.DataFrame,
    cat_mda: pd.DataFrame,
    topC: int = 10,           # 글로벌 상위 매체
    add_random: int = 2,      # 랜덤 탐색
    cat_topM: int = 5,        # 카테고리 상위 매체
    min_active_clk: int = 200,# 활동 필터
    min_active_conv: int = 20,
    rng_seed: int = 42,
    ad_ids_filter: Optional[np.ndarray] = None,  # 학습용 광고 제한
    max_per_ad: int = 20                         # ad별 최대 후보 수
) -> pd.DataFrame:
    # 비활성 매체 컷
    mda_active = mda_all[
        (mda_all["mda_clk_global"]  >= min_active_clk) |
        (mda_all["mda_conv_global"] >= min_active_conv)
    ].copy()
    if mda_active.empty:
        mda_active = mda_all.copy()

    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)

    # 카테고리 상위 캐시
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True, False])
                      .groupby("ads_category")
                      .head(cat_topM)[["ads_category","mda_idx"]].copy())
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c, g in cat_top.groupby("ads_category")}

    rng = np.random.RandomState(rng_seed)
    # 광고 대상 제한
    if ad_ids_filter is not None and len(ad_ids_filter) > 0:
        ad_ids = np.array(ad_ids_filter, dtype=int)
    else:
        ad_ids = ads_list["ads_idx"].dropna().astype(int).unique()

    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs = []
    for ad in ad_ids:
        base = []
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top:
            base.extend(cat2top[cat])  # 카테고리 상위 매체
        base.extend(mda_ids_global[:min(topC, len(mda_ids_global))])  # 글로벌 상위 매체
        base = list(dict.fromkeys(base))  # 중복 제거
        # 랜덤 탐색
        remain = np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain) > 0 and add_random > 0:
            base.extend(rng.choice(remain, size=min(add_random, len(remain)), replace=False).tolist())
        # ad별 최대 후보 수 제한
        if len(base) > max_per_ad:
            base = base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]

    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} "
          f"| topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 학습 데이터(허용 피처만 생성) — 속성 전용 (+선택: media_portfolio)
# -------------------------------
FEAT_ALLOW = [
    # 매체 전반(OBS 집계)
    "mda_clk_global", "mda_conv_global", "mda_cvr_global", "mda_pop_norm",

    # 카테고리 × 매체 적합도(OBS 집계)
    "cat_cvr", "cat_share",

    # 광고 가격/속성(ads_list)
    "price_reward_log", "price_contract_log", "price_ratio",

    # 도메인 / OS 적합도
    "dom_cvr", "dom_share", "os_cvr", "os_share",

    # media_portfolio(있으면 사용, 없으면 0)
    "mda_size", "active_ratio", "day_ratio", "weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame,
    tgt: pd.DataFrame,
    ads_list: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None  # 선택 병합
):
    # 학습 대상 광고 제한
    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt,
                                        max_train_ads=max_train_ads)

    actx = build_attr_context(obs, ads_list)
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
    dom_mda, os_mda = actx.get("dom_mda"), actx.get("os_mda")

    # 후보 생성(축소)
    cand = create_candidates(
        ads_list=ads_list,
        mda_all=mda_all,
        cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # 매체 피처(전역 허용분)
    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    # 광고 피처 + 도메인/OS
    ad_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx").copy()
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain","site_domain","publisher_domain"])
    ad_meta = _ensure_col(ad_meta, "os_type",    ["os_type","device_os","os"])
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","os_type",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    # 카테고리×매체
    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0, "cat_share":0.0})

    # 도메인×매체
    if dom_mda is not None:
        dom_mda = dom_mda.copy()
        dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
        df["ads_domain"] = _norm_cat(df["ads_domain"])
        df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                      on=["mda_idx","ads_domain"], how="left")
    else:
        df["dom_cvr"] = 0.0; df["dom_share"] = 0.0

    # OS×매체
    if os_mda is not None:
        os_mda = os_mda.copy()
        os_mda["os_type"] = _norm_cat(os_mda["os_type"])
        df["os_type"] = _norm_cat(df["os_type"])
        df = df.merge(os_mda[["mda_idx","os_type","os_cvr","os_share"]],
                      on=["mda_idx","os_type"], how="left")
    else:
        df["os_cvr"] = 0.0; df["os_share"] = 0.0

    df[["dom_cvr","dom_share","os_cvr","os_share"]] = df[["dom_cvr","dom_share","os_cvr","os_share"]].fillna(0.0)

    # (선택) media_portfolio 병합
    if media_portfolio is not None:
        mp = media_portfolio.copy()
        merge_cols = ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"]
        use_cols = [c for c in merge_cols if c in mp.columns]
        if "mda_idx" in use_cols:
            df = df.merge(mp[use_cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns:
            df[c] = 0.0

    # 라벨(TGT 전환수) — 피처엔 사용하지 않음
    tgt_g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
               .agg(tgt_turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    df = df.merge(tgt_g, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})

    # relevance bin (0=neg, 1/2/3=pos bins)
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum() > 0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int) + 1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 피처 매트릭스 / 다운샘플 / 학습
# -------------------------------
LABEL_GAIN = [0.0, 1.0, 4.0, 12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c] = 0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"] > 0]; neg = g[g["tgt_turn"] <= 0]
        if len(pos) == 0: continue
        cap = len(pos) * ratio
        if len(neg) > cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos, neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    w = np.ones(len(train_df), dtype=float)
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
        learning_rate=0.05, n_estimators=1200, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=w)
    return ranker, feat_cols

# -------------------------------
# 7) 예측(배치, 상한) — 전 mda에서 스코어 후 Top-K
# -------------------------------
def predict_all_mda_attr_only_fast_batched(
    ranker, feat_cols, actx, ads_list, K=20,
    batch_size: int = 5000,
    max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None
) -> Dict[int, List[Tuple[int,float]]]:
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
    dom_mda, os_mda = actx.get("dom_mda"), actx.get("os_mda")

    mda_active = mda_all[(mda_all["mda_clk_global"] >= 200) | (mda_all["mda_conv_global"] >= 20)]
    if mda_active.empty: mda_active = mda_all
    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"] = mda_active["mda_idx"].astype(int)

    # (선택) media_portfolio 병합
    if media_portfolio is not None:
        mp = media_portfolio.copy()
        merge_cols = ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"]
        use_cols = [c for c in merge_cols if c in mp.columns]
        if "mda_idx" in use_cols:
            mda_active = mda_active.merge(mp[use_cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns:
            mda_active[c] = 0.0

    # 광고 메타 + 도메인/OS
    ads_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None:
        ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain","site_domain","publisher_domain"])
    ads_meta = _ensure_col(ads_meta, "os_type",    ["os_type","device_os","os"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["os_type"]    = _norm_cat(ads_meta["os_type"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] /
                                      ads_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")

    # dom/os lookup 준비
    if dom_mda is not None:
        dom_mda_local = dom_mda.copy()
        dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])
    else:
        dom_mda_local = None

    if os_mda is not None:
        os_mda_local = os_mda.copy()
        os_mda_local["os_type"] = _norm_cat(os_mda_local["os_type"])
    else:
        os_mda_local = None

    preds: Dict[int, List[Tuple[int,float]]] = {}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        # cross join: ad_batch × mda_active
        grid = ad_batch[["ads_idx","ads_category","ads_domain","os_type",
                         "price_reward_log","price_contract_log","price_ratio"]].merge(
            mda_active, how="cross"
        )
        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left") \
                   .fillna({"cat_cvr":0.0, "cat_share":0.0})

        # 도메인/OS 병합
        if dom_mda_local is not None:
            grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                              on=["mda_idx","ads_domain"], how="left")
        else:
            grid["dom_cvr"]=0.0; grid["dom_share"]=0.0

        if os_mda_local is not None:
            grid = grid.merge(os_mda_local[["mda_idx","os_type","os_cvr","os_share"]],
                              on=["mda_idx","os_type"], how="left")
        else:
            grid["os_cvr"]=0.0; grid["os_share"]=0.0

        grid[["dom_cvr","dom_share","os_cvr","os_share"]] = grid[["dom_cvr","dom_share","os_cvr","os_share"]].fillna(0.0)

        X = ensure_feature_matrix(grid, feat_cols)
        scores = ranker.predict(X)
        grid = grid[["ads_idx","mda_idx"]].copy()
        grid["score"] = scores

        for ad, g in grid.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            preds[int(ad)] = list(zip(top["mda_idx"].astype(int).tolist(),
                                      top["score"].astype(float).tolist()))
        print(f"[Predict] processed ads {i+len(ad_batch)}/{len(ad_ids)}")

    return preds

# -------------------------------
# 8) 간단 평가
# -------------------------------
def evaluate_simple(preds_dict, gt_dict, k=10):
    P, R, H = [], [], []
    for ad, lst in preds_dict.items():
        if ad not in gt_dict: continue
        pred = [m for m,_ in lst[:k]]
        true = set(gt_dict[ad])
        if not true: continue
        inter = len(set(pred) & true)
        P.append(inter / max(1, len(pred)))
        R.append(inter / len(true))
        H.append(1.0 if inter > 0 else 0.0)
    return {"precision@%d"%k: float(np.mean(P)) if P else 0.0,
            "recall@%d"%k:    float(np.mean(R)) if R else 0.0,
            "hit_rate@%d"%k:  float(np.mean(H)) if H else 0.0,
            "evaluated_ads":  len(P)}

# ==============================================
# ==============   메인 플로우   ===============
# ==============================================
# 요구: ads_time, ads_list (필수)
# 선택: media_portfolio (있으면 피처 보강)

# (A) 윈도우 — 요청하신 날짜로 고정
obs, tgt, start, end, cut = split_obs_tgt_by_dates(
    ads_time,
    obs_start="2025-07-25",   # 관찰 시작일
    tgt_start="2025-08-20"    # 타깃 시작일
)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  "
      f"TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (B) 학습 데이터 (강한 축소 파라미터로 안전하게)
train_df, actx = prepare_training_attr_only(
    obs, tgt, ads_list,
    topC=10, add_random=2, cat_topM=5,            # 후보 소형화
    min_active_clk=200, min_active_conv=20,       # 비활성 매체 컷
    max_train_ads=20000, min_pos_in_tgt=1
)
print("train_df size:", len(train_df))

# (C) 다운샘플 → 학습
train_df = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols = train_attr_ranker(train_df)
print("feat_cols used:", feat_cols)

# (D) 예측(배치 + 상한) & 평가
preds = predict_all_mda_attr_only_fast_batched(
    ranker, feat_cols, actx, ads_list, K=20,
    batch_size=5000,       # 환경에 따라 2000~10000 조절
    max_predict_ads=20000  # 전량 필요시 None
)
gt = tgt.groupby("ads_idx")["mda_idx"].apply(list).to_dict()
print("Eval:", evaluate_simple(preds, gt, k=10))


NameError: name 'ads_time' is not defined

In [42]:
feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
print("사용된 피처:", feat_cols)

# 샘플 데이터 확인
train_df[feat_cols].head(30)


사용된 피처: ['mda_clk_global', 'mda_conv_global', 'mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'cat_share', 'price_reward_log', 'price_contract_log', 'price_ratio', 'dom_cvr', 'dom_share', 'os_cvr', 'os_share', 'mda_size', 'active_ratio', 'day_ratio', 'weekend_ratio']


,mda_clk_global,mda_conv_global,mda_cvr_global,mda_pop_norm,cat_cvr,cat_share,price_reward_log,price_contract_log,price_ratio,dom_cvr,dom_share,os_cvr,os_share,mda_size,active_ratio,day_ratio,weekend_ratio
0,71,43,0.605634,-0.247868,1.000000,0.023256,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,3,0.600000,-0.249776,1.000000,0.333333,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,1.000000,-0.249871,1.000000,1.000000,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,1.000000,-0.249871,1.000000,1.000000,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,418764,240354,0.573961,11.209750,0.123908,0.001652,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,314437,90485,0.287768,4.064252,0.028973,0.008156,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,175706,90276,0.513790,4.054287,0.193106,0.005461,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,124140,59528,0.479523,2.588275,0.173290,0.016849,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,167445,57620,0.344113,2.497305,0.048106,0.015342,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,90599,52532,0.579830,2.254717,0.146378,0.005539,7.090910,7.313887,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# --- A1. 예측 시 사용된 활성 매체 집합 추출 ---
mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
mda_active = mda_all[(mda_all["mda_clk_global"] >= 200) | (mda_all["mda_conv_global"] >= 20)]
if mda_active.empty: mda_active = mda_all
active_set = set(mda_active["mda_idx"].astype(int).tolist())

# --- A2. GT (TGT) 구성 ---
gt_dict = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: set(pd.to_numeric(s, errors="coerce").dropna().astype(int))).to_dict()

# --- A3. 후보 상한(coverage) 측정: GT 중 active_set 안에 존재하는 비율 ---
cover_rates = []
for ad, true_set in gt_dict.items():
    if not true_set: 
        continue
    inter = len(true_set & active_set)
    cover_rates.append(inter / len(true_set))

print(
    "[Coverage] mean=%.3f | median=%.3f | pct_full_covered=%.1f%%" % (
        np.mean(cover_rates) if cover_rates else 0.0,
        np.median(cover_rates) if cover_rates else 0.0,
        100.0 * np.mean([c == 1.0 for c in cover_rates]) if cover_rates else 0.0
    )
)


[Coverage] mean=0.992 | median=1.000 | pct_full_covered=97.6%


In [25]:
# ==============================================
#  Ad × Media 추천 (Attribute-only, Cold-start Safe)
#  - 과거 "광고×매체" 페어 성과/CF 전부 배제
#  - 허용 피처: mda_* / cat_* / price_* / (옵션) dom_*, os_* / (옵션) media_portfolio
#  - 윈도우: OBS=학습 컨텍스트, TGT=평가 구간 (날짜 고정)
#  - 목표: Precision 소폭 상향 (neg:pos=2:1 다운샘플 + Binary 분류 + K×th 스윕)
# ==============================================

import numpy as np
import pandas as pd
from typing import Optional, List, Dict, Tuple
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import lightgbm as lgb

# -------------------------------
# 공통 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str = "ads_category") -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s):
    return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    if col is None:
        df[out_col] = "__na__"
    else:
        df[out_col] = _norm_cat(df[col])
    return df

# -------------------------------
# 1) 윈도우 (날짜 직접 지정)
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6  # tgt_end 미지정 시, 최대 7일(0~6) 사용
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")

    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()

    if obs_end is None:
        obs_end = (tgt_start - pd.Timedelta(days=1)).normalize()
    else:
        obs_end = pd.to_datetime(obs_end).normalize()

    last_date = t["date"].max().normalize()
    if tgt_end is None:
        tgt_end = min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize())
    else:
        tgt_end = pd.to_datetime(tgt_end).normalize()

    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()

    month_start = obs_start
    month_end   = tgt_end
    cutoff      = (tgt_start - pd.Timedelta(days=1)).normalize()
    return obs, tgt, month_start, month_end, cutoff

# -------------------------------
# 2) 속성 컨텍스트(OBS만 사용)
# -------------------------------
def build_attr_context(obs: pd.DataFrame, ads_list: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # 매체 전반 (OBS)
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"),
                      mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0, np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체 (OBS)
    al = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])

    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"] / cat_mda["clk"].replace(0, np.nan)).fillna(0.0)

    # 각 mda에서 카테고리 전환 비중(cat_share)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"] /
                            cat_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)

    # --- 도메인 / OS 적합도 (OBS 로그에 있을 경우만) ---
    dom_col = _pick_col(t.columns, ["req_domain","domain","site_domain","publisher_domain"])
    os_col  = _pick_col(t.columns, ["os_type","device_os","os"])

    dom_mda = None
    if dom_col is not None:
        t_dom = t.copy()
        t_dom[dom_col] = _norm_cat(t_dom[dom_col])
        dom_mda = (t_dom.groupby(["mda_idx", dom_col], as_index=False)
                        .agg(clk=("clk","sum"), conv=("conv","sum")))
        dom_mda["dom_cvr"] = (dom_mda["conv"] / dom_mda["clk"].replace(0, np.nan)).fillna(0.0)
        mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
        dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
        dom_mda["dom_share"] = (dom_mda["conv"] / dom_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)
        dom_mda = dom_mda.rename(columns={dom_col: "ads_domain"})

    os_mda = None
    if os_col is not None:
        t_os = t.copy()
        t_os[os_col] = _norm_cat(t_os[os_col])
        os_mda = (t_os.groupby(["mda_idx", os_col], as_index=False)
                        .agg(clk=("clk","sum"), conv=("conv","sum")))
        os_mda["os_cvr"] = (os_mda["conv"] / os_mda["clk"].replace(0, np.nan)).fillna(0.0)
        mda_tot3 = os_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
        os_mda = os_mda.merge(mda_tot3, on="mda_idx", how="left")
        os_mda["os_share"] = (os_mda["conv"] / os_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)
        os_mda = os_mda.rename(columns={os_col: "os_type"})

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda, "os_mda": os_mda}

# -------------------------------
# 3) 학습 대상 광고 선별 (TGT에서 양성 존재 + 상한)
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000,
                            rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1, int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성(속성 기반, 강력 축소)
# -------------------------------
def create_candidates(
    ads_list: pd.DataFrame,
    mda_all: pd.DataFrame,
    cat_mda: pd.DataFrame,
    topC: int = 10,
    add_random: int = 2,
    cat_topM: int = 5,
    min_active_clk: int = 200,
    min_active_conv: int = 20,
    rng_seed: int = 42,
    ad_ids_filter: Optional[np.ndarray] = None,
    max_per_ad: int = 20
) -> pd.DataFrame:
    # 비활성 매체 컷
    mda_active = mda_all[
        (mda_all["mda_clk_global"]  >= min_active_clk) |
        (mda_all["mda_conv_global"] >= min_active_conv)
    ].copy()
    if mda_active.empty:
        mda_active = mda_all.copy()

    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)

    # 카테고리 상위 캐시
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True, False])
                      .groupby("ads_category")
                      .head(cat_topM)[["ads_category","mda_idx"]].copy())
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c, g in cat_top.groupby("ads_category")}

    rng = np.random.RandomState(rng_seed)
    # 광고 대상
    if ad_ids_filter is not None and len(ad_ids_filter) > 0:
        ad_ids = np.array(ad_ids_filter, dtype=int)
    else:
        ad_ids = ads_list["ads_idx"].dropna().astype(int).unique()

    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs = []
    for ad in ad_ids:
        base = []
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top:
            base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC, len(mda_ids_global))])
        base = list(dict.fromkeys(base))
        remain = np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain) > 0 and add_random > 0:
            base.extend(rng.choice(remain, size=min(add_random, len(remain)), replace=False).tolist())
        if len(base) > max_per_ad:
            base = base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]

    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} "
          f"| topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 학습 데이터(허용 피처만 생성)
# -------------------------------
FEAT_ALLOW = [
    # 매체 전반(OBS 집계)
    "mda_clk_global", "mda_conv_global", "mda_cvr_global", "mda_pop_norm",
    # 카테고리 × 매체
    "cat_cvr", "cat_share",
    # 광고 가격/속성
    "price_reward_log", "price_contract_log", "price_ratio",
    # 도메인/OS 적합도
    "dom_cvr", "dom_share", "os_cvr", "os_share",
    # (옵션) media_portfolio
    "mda_size", "active_ratio", "day_ratio", "weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame,
    tgt: pd.DataFrame,
    ads_list: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None
):
    # 학습 대상 광고 제한
    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt,
                                        max_train_ads=max_train_ads)

    actx = build_attr_context(obs, ads_list)
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
    dom_mda, os_mda = actx.get("dom_mda"), actx.get("os_mda")

    # 후보 생성
    cand = create_candidates(
        ads_list=ads_list,
        mda_all=mda_all,
        cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # 매체 피처
    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    # 광고 피처 + 도메인/OS
    ad_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx").copy()
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain","site_domain","publisher_domain"])
    ad_meta = _ensure_col(ad_meta, "os_type",    ["os_type","device_os","os"])
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)
    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","os_type",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    # 카테고리×매체
    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0, "cat_share":0.0})

    # 도메인×매체
    if dom_mda is not None:
        dom_mda = dom_mda.copy()
        dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
        df["ads_domain"] = _norm_cat(df["ads_domain"])
        df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                      on=["mda_idx","ads_domain"], how="left")
    else:
        df["dom_cvr"] = 0.0; df["dom_share"] = 0.0

    # OS×매체
    if os_mda is not None:
        os_mda = os_mda.copy()
        os_mda["os_type"] = _norm_cat(os_mda["os_type"])
        df["os_type"] = _norm_cat(df["os_type"])
        df = df.merge(os_mda[["mda_idx","os_type","os_cvr","os_share"]],
                      on=["mda_idx","os_type"], how="left")
    else:
        df["os_cvr"] = 0.0; df["os_share"] = 0.0

    df[["dom_cvr","dom_share","os_cvr","os_share"]] = df[["dom_cvr","dom_share","os_cvr","os_share"]].fillna(0.0)

    # (옵션) media_portfolio 병합
    if media_portfolio is not None:
        mp = media_portfolio.copy()
        merge_cols = ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"]
        use_cols = [c for c in merge_cols if c in mp.columns]
        if "mda_idx" in use_cols:
            df = df.merge(mp[use_cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns:
            df[c] = 0.0

    # 라벨 = TGT 전환수(회귀/분류/랭킹 어디서든 쓸 수 있게 보관)
    tgt_g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
               .agg(tgt_turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    df = df.merge(tgt_g, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})

    # 참고용 relevance(랭커 쓸 때 활용 가능)
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum() > 0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int) + 1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 피처 매트릭스 / 다운샘플 / 학습
# -------------------------------
def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c] = 0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

# (패치) 안전한 다운샘플 함수: tgt_turn만 사용
def downsample_negatives_per_ad(df, ratio=2, random_state=42):
    if "tgt_turn" not in df.columns:
        raise ValueError("train_df에 'tgt_turn' 컬럼이 없습니다.")
    t = df.copy()
    t["tgt_turn"] = pd.to_numeric(t["tgt_turn"], errors="coerce").fillna(0)
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in t.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"] > 0]
        neg = g[g["tgt_turn"] <= 0]
        if len(pos) == 0:
            continue
        cap = len(pos) * ratio
        if len(neg) > cap:
            neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos, neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else t

# -------------------------------
# 7) 모든 ad×mda raw score 생성 (분류기/랭커 공통)
# -------------------------------
def predict_scores_full(model, feat_cols, actx, ads_list, batch_size=5000, max_predict_ads=20000):
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
    dom_mda, os_mda = actx.get("dom_mda"), actx.get("os_mda")

    # 활성 매체
    mda_active = mda_all[(mda_all["mda_clk_global"] >= 200) | (mda_all["mda_conv_global"] >= 20)]
    if mda_active.empty:
        mda_active = mda_all
    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"] = mda_active["mda_idx"].astype(int)

    # 광고 메타
    ads_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]] \
                    .drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None:
        ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain","site_domain","publisher_domain"])
    ads_meta = _ensure_col(ads_meta, "os_type",    ["os_type","device_os","os"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["os_type"]    = _norm_cat(ads_meta["os_type"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] /
                                      ads_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")

    dom_mda_local = None
    if dom_mda is not None:
        dom_mda_local = dom_mda.copy()
        dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])
    os_mda_local = None
    if os_mda is not None:
        os_mda_local = os_mda.copy()
        os_mda_local["os_type"] = _norm_cat(os_mda_local["os_type"])

    rows = []
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","os_type",
                         "price_reward_log","price_contract_log","price_ratio"]].merge(
            mda_active, how="cross"
        )
        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left") \
                   .fillna({"cat_cvr":0.0, "cat_share":0.0})

        if dom_mda_local is not None:
            grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                              on=["mda_idx","ads_domain"], how="left")
        else:
            grid["dom_cvr"]=0.0; grid["dom_share"]=0.0
        if os_mda_local is not None:
            grid = grid.merge(os_mda_local[["mda_idx","os_type","os_cvr","os_share"]],
                              on=["mda_idx","os_type"], how="left")
        else:
            grid["os_cvr"]=0.0; grid["os_share"]=0.0

        grid[["dom_cvr","dom_share","os_cvr","os_share"]] = grid[["dom_cvr","dom_share","os_cvr","os_share"]].fillna(0.0)

        X = ensure_feature_matrix(grid, feat_cols)
        try:
            scores = model.predict_proba(X)[:,1]   # 분류기
        except Exception:
            scores = model.predict(X)             # 랭커 등
        grid["score"] = scores
        rows.append(grid[["ads_idx","mda_idx","score"]])

        print(f"[predict_scores_full] processed ads {i+len(ad_batch)}/{len(ad_ids)}")

    return pd.concat(rows, ignore_index=True)

# -------------------------------
# 8) 평가 유틸
# -------------------------------
def eval_from_scores(score_df, gt_dict, K=10, score_min=None):
    P, R, H = [], [], []
    for ad, g in score_df.groupby("ads_idx"):
        if ad not in gt_dict:
            continue
        gg = g if score_min is None else g[g["score"] >= score_min]
        if gg.empty:
            continue
        top = gg.nlargest(K, "score")["mda_idx"].astype(int).tolist()
        true = set(gt_dict[ad])
        if not true:
            continue
        inter = len(set(top) & true)
        P.append(inter / max(1, len(top)))
        R.append(inter / len(true))
        H.append(1.0 if inter > 0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0,
            int(len(P)))

def evaluate_simple_from_preds(preds_dict, gt_dict, k):
    P, R, H = [], [], []
    for ad, lst in preds_dict.items():
        if ad not in gt_dict:
            continue
        pred = [m for m,_ in lst[:k]]
        true = set(gt_dict[ad])
        if not true:
            continue
        inter = len(set(pred) & true)
        P.append(inter / max(1, len(pred)))
        R.append(inter / len(true))
        H.append(1.0 if inter > 0 else 0.0)
    return {"precision@%d"%k: float(np.mean(P)) if P else 0.0,
            "recall@%d"%k:    float(np.mean(R)) if R else 0.0,
            "hit_rate@%d"%k:  float(np.mean(H)) if H else 0.0,
            "evaluated_ads":  int(len(P))}

# ==============================================
# ==============   메인 플로우   ===============
# ==============================================
# 요구: ads_time, ads_list (필수)
# 선택: media_portfolio (있으면 피처 보강)

# (A) 윈도우 — 요청 날짜
obs, tgt, start, end, cut = split_obs_tgt_by_dates(
    ads_time,
    obs_start="2025-07-25",
    tgt_start="2025-08-20"
)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (B) 학습 데이터 생성
train_df, actx = prepare_training_attr_only(
    obs, tgt, ads_list,
    topC=7, add_random=1, cat_topM=4,           # 최근에 성능 괜찮았던 설정
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1
)
print("train_df size:", len(train_df), "| pos rate:", (train_df["tgt_turn"]>0).mean())

# (C) 다운샘플 → Binary 분류기 학습 (neg:pos=2:1)
train_df2 = downsample_negatives_per_ad(train_df, ratio=2, random_state=42)
feat_cols = [c for c in FEAT_ALLOW if c in train_df2.columns]
X_tr = ensure_feature_matrix(train_df2, feat_cols)
y_tr = (pd.to_numeric(train_df2["tgt_turn"], errors="coerce").fillna(0) > 0).astype(int)

clf = lgb.LGBMClassifier(
    objective="binary", metric="auc",
    learning_rate=0.05, n_estimators=1200, num_leaves=63,
    min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
    class_weight="balanced", random_state=42, n_jobs=-1, verbosity=-1
)
clf.fit(X_tr, y_tr)
print("[Train] X:", X_tr.shape, "| pos rate:", y_tr.mean())

# (D) 점수 테이블 (ad × mda)
score_df = predict_scores_full(clf, feat_cols, actx, ads_list,
                               batch_size=5000, max_predict_ads=20000)
print("[score_df] rows:", len(score_df), "| ads:", score_df['ads_idx'].nunique(), "| mda uniq:", score_df['mda_idx'].nunique())
print(score_df['score'].describe(percentiles=[.5,.8,.9,.95,.98,.99]))

# (E) GT & Coverage 참고 (옵션)
gt_dict = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: set(pd.to_numeric(s, errors="coerce").dropna().astype(int))).to_dict()
mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
mda_active = mda_all[(mda_all["mda_clk_global"] >= 200) | (mda_all["mda_conv_global"] >= 20)]
if mda_active.empty: mda_active = mda_all
active_set = set(mda_active["mda_idx"].astype(int).tolist())
cover_rates = []
for ad, true_set in gt_dict.items():
    if not true_set: continue
    inter = len(true_set & active_set)
    cover_rates.append(inter / len(true_set))
if cover_rates:
    print("[Coverage] mean=%.3f | median=%.3f | pct_full_covered=%.1f%%" %
          (np.mean(cover_rates), np.median(cover_rates), 100.0*np.mean([c==1.0 for c in cover_rates])))

# (F) K × threshold 스윕 (precision 최적)
grid, best = [], None
K_list  = [3, 5, 10]
TH_list = [None, 0.05, 0.08, 0.10, 0.12, 0.15]

for K in K_list:
    for th in TH_list:
        p, r, h, n = eval_from_scores(score_df, gt_dict, K=K, score_min=th)
        grid.append((K, th, p, r, h, n))
        if (best is None) or (p > best[2]):
            best = (K, th, p, r, h, n)

print("\n=== Grid Top-10 by Precision ===")
for row in sorted(grid, key=lambda x: x[2], reverse=True)[:10]:
    print(f"K={row[0]}, th={row[1]}, P@K={row[2]:.3f}, R@K={row[3]:.3f}, Hit@K={row[4]:.3f}, N={row[5]}")

print("\n=== Best by Precision ===")
print(f"K={best[0]}, th={best[1]}, P@K={best[2]:.3f}, R@K={best[3]:.3f}, Hit@K={best[4]:.3f}, N={best[5]}")

# (G) 베스트 조합으로 최종 preds & 평가
K_star, th_star = best[0], best[1]
preds_best = {}
for ad, g in score_df.groupby("ads_idx", sort=False):
    gg = g if th_star is None else g[g["score"] >= th_star]
    if gg.empty:
        preds_best[int(ad)] = []
        continue
    top = gg.nlargest(K_star, "score")[["mda_idx","score"]]
    preds_best[int(ad)] = list(zip(top["mda_idx"].astype(int).tolist(),
                                   top["score"].astype(float).tolist()))

final_eval = evaluate_simple_from_preds(preds_best, gt_dict, k=K_star)
print("\n=== Final Eval with Best (K, threshold) ===")
print(final_eval)


[Window] OBS: 2025-07-25 → 2025-08-19  |  TGT: 2025-08-20 → 2025-08-25
[Candidates] pairs=20,368 | ads=1,848 | active_mda=148 | topC=7, rnd=1, cat_topM=4, per_ad≤20
train_df size: 20368 | pos rate: 0.08361154752553024
[Train] X: (5103, 17) | pos rate: 0.3337252596511856
[predict_scores_full] processed ads 5000/20000
[predict_scores_full] processed ads 10000/20000
[predict_scores_full] processed ads 15000/20000
[predict_scores_full] processed ads 20000/20000
[score_df] rows: 2960000 | ads: 20000 | mda uniq: 148
count    2.960000e+06
mean     1.643028e-01
std      2.614082e-01
min      2.437749e-07
50%      2.134792e-02
80%      3.118332e-01
90%      6.173666e-01
95%      8.059786e-01
98%      9.344098e-01
99%      9.674703e-01
max      9.999533e-01
Name: score, dtype: float64
[Coverage] mean=0.992 | median=1.000 | pct_full_covered=97.6%

=== Grid Top-10 by Precision ===
K=5, th=0.15, P@K=0.237, R@K=0.385, Hit@K=0.534, N=88
K=5, th=0.05, P@K=0.236, R@K=0.385, Hit@K=0.534, N=88
K=5, th=0.

In [26]:
# TGT 구간에 등장한 광고 목록 (중복 제거)
tgt_ads = tgt["ads_idx"].dropna().unique()
print("TGT 광고 개수:", len(tgt_ads))
print("TGT 광고 목록:", tgt_ads[:50])  # 앞에서 50개만 예시 출력


TGT 광고 개수: 2456
TGT 광고 목록: [ 12923  13927  13928  14074  14405  16523  16528  16529  16895  20256
  20257  20737  21037  21038  22232  22498  24944  26416  27576  28025
  28051  28052  28302  28781  29699  49983  49985  55718  57056  72966
  72967  73878  80973  86708  93023  95751  96590 103599 119866 119999
 127906 128634 131204 131233 133706 135570 139969 140001 140831 163200]


In [27]:
# ads_list에 있는 광고 이름/카테고리랑 join
tgt_ads_meta = ads_list[ads_list["ads_idx"].isin(tgt_ads)][
    ["ads_idx","ads_name","ads_category","ads_reward_price","ads_contract_price"]
]
tgt_ads_meta.head(20)


,ads_idx,ads_name,ads_category,ads_reward_price,ads_contract_price
5342,6985,DB손해보험 다이렉트,7,2300,2700
8287,9935,숨어있는 휴면포인트 조회,8,1200,1500
8332,9982,강원일보 네이버 뉴스,3,160,230
8354,10004,원키퍼,10,1120,1500
8837,10500,판타지타운,5,3000,4000
9075,10742,스치듯 로얄리스트 고체향수 5g,11,1900,2973
9078,10745,스치듯 포레스트 고체향수 15g,11,3140,4909
9086,10753,매드라운드 에고이즘 오드퍼퓸 30ml,11,9920,15273
9385,11059,파이널삼국지2,5,200,300
9446,11123,프린세스테일,6,429,0


In [28]:
# TGT 구간 광고별 전환수 집계
tgt_perf = (
    tgt.groupby("ads_idx", as_index=False)
       .agg(clicks=("rpt_time_clk","sum"),
            turns=("rpt_time_turn","sum"))
       .sort_values("turns", ascending=False)
)

# 메타데이터 붙이기
tgt_perf = tgt_perf.merge(
    ads_list[["ads_idx","ads_name","ads_category"]],
    on="ads_idx", how="left"
)
tgt_perf.head(20)


,ads_idx,clicks,turns,ads_name,ads_category
0,445047,192051,117466,K데몬헌터스,2
1,445046,129962,65168,K데몬헌터스,2
2,444350,119491,50231,조조의 꿈,1
3,444349,96064,47914,조조의 꿈,2
4,446261,17708,15596,마루는 강쥐:나 카페 사장됐다? 짱이지!,2
5,440167,64885,13662,[오픈하기] KB차차차,1
6,445907,15365,13046,영웅줍줍 장비줍줍,2
7,445514,23871,10099,[원스토어] 심연의 검,2
8,443782,13206,10060,디바인엣지: 소녀와 마신의 이야기,2
9,445048,21960,9097,[원스토어] K데몬헌터스,2


In [31]:
# (1) Top-K 리스트가 비어있지 않은 광고만 추출
pred_non_empty = {ad: lst for ad, lst in preds.items() if len(lst) > 0}

# (2) TGT 구간 광고
tgt_ads = tgt["ads_idx"].dropna().unique()

# (3) 예측 광고
pred_ads = list(pred_non_empty.keys())

# (4) 교집합 (둘 다에 있는 광고)
common_ads = np.intersect1d(tgt_ads, pred_ads)

print("TGT 광고 수:", len(tgt_ads))
print("예측 광고 수:", len(pred_ads))
print("교집합 광고 수:", len(common_ads))
print("교집합 광고 ID 샘플:", common_ads[:20])  # 앞 20개만


TGT 광고 수: 2456
예측 광고 수: 20000
교집합 광고 수: 88
교집합 광고 ID 샘플: [ 6985  9935  9982 10004 10500 10742 10745 10753 11059 11123 11124 12923
 13209 13927 13928 13988 14012 14074 14134 14196]


In [38]:
K = 5  # Top-K
ad_id = 11059  # 특정 광고 ID 예시

# 모델이 예측한 상위 K 매체
pred_topk = [m for m,_ in preds.get(ad_id, [])[:K]]

# 실제 전환이 있었던 매체 (TGT에서 추출)
true_mda = set(gt.get(ad_id, []))

print("광고 ID:", ad_id)
print("예측 Top-K:", pred_topk)
print("실제 전환 매체:", true_mda)

# 맞춘 매체 vs 놓친 매체
hits = set(pred_topk) & true_mda
miss_pred = set(pred_topk) - true_mda
miss_true = true_mda - set(pred_topk)

print("맞춘 매체:", hits)
print("추천했지만 실패:", miss_pred)
print("놓친 실제 매체:", miss_true)


광고 ID: 11059
예측 Top-K: [270, 482, 562, 22, 14]
실제 전환 매체: {643, 398, 337, 761, 30, 281, 766}
맞춘 매체: set()
추천했지만 실패: {482, 14, 270, 562, 22}
놓친 실제 매체: {643, 281, 398, 766, 337, 761, 30}


In [39]:
K = 10  # Top-K
ad_id = 11059  # 특정 광고 ID 예시

# 모델이 예측한 상위 K 매체
pred_topk = [m for m,_ in preds.get(ad_id, [])[:K]]

# 실제 전환이 있었던 매체 (TGT에서 추출)
true_mda = set(gt.get(ad_id, []))

print("광고 ID:", ad_id)
print("예측 Top-K:", pred_topk)
print("실제 전환 매체:", true_mda)

# 맞춘 매체 vs 놓친 매체
hits = set(pred_topk) & true_mda
miss_pred = set(pred_topk) - true_mda
miss_true = true_mda - set(pred_topk)

print("맞춘 매체:", hits)
print("추천했지만 실패:", miss_pred)
print("놓친 실제 매체:", miss_true)

# ======================
# 평가 지표 계산 (단일 광고)
# ======================
if len(pred_topk) > 0 and len(true_mda) > 0:
    precision = len(hits) / len(pred_topk)
    recall = len(hits) / len(true_mda)
    hit_rate = 1.0 if len(hits) > 0 else 0.0
else:
    precision, recall, hit_rate = 0.0, 0.0, 0.0

print(f"Precision@{K}: {precision:.3f}")
print(f"Recall@{K}:    {recall:.3f}")
print(f"HitRate@{K}:   {hit_rate:.3f}")


광고 ID: 11059
예측 Top-K: [270, 482, 562, 22, 14, 56, 667, 371, 563, 540]
실제 전환 매체: {643, 398, 337, 761, 30, 281, 766}
맞춘 매체: set()
추천했지만 실패: {482, 14, 270, 562, 371, 563, 22, 56, 667, 540}
놓친 실제 매체: {643, 281, 398, 766, 337, 761, 30}
Precision@10: 0.000
Recall@10:    0.000
HitRate@10:   0.000


In [ ]:
def search_and_predict_ad(ad_id_or_str, obs, ads_list, ranker, feat_cols, topk=10):
    """
    단일 광고(ad_id)에 대해 전 매체(활성 매체 위주) 스코어링 후 TOP-K 반환
    반환: DataFrame [ads_idx, mda_idx, score, mda_cvr_global, mda_pop_norm, cat_cvr]
    """
    # 1) 입력 파싱 & 컨텍스트 준비
    ad_id = int(str(ad_id_or_str).strip())
    actx = build_attr_context(obs, ads_list)
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]

    # 2) 예측 시에도 너무 느려지지 않도록 '활성 매체'만 우선 사용 (없으면 전체 사용)
    mda_active = mda_all[(mda_all["mda_clk_global"] >= 200) | (mda_all["mda_conv_global"] >= 20)]
    if mda_active.empty:
        mda_active = mda_all
    mda_active = mda_active[["mda_idx","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"] = mda_active["mda_idx"].astype(int)

    # 3) 광고 메타 (카테고리/가격 → 피처)
    meta = ads_list.loc[ads_list["ads_idx"] == ad_id,
                        ["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx")
    if meta.empty:
        raise ValueError(f"ads_list에 ads_idx={ad_id} 광고가 없습니다.")
    meta = _norm_cat_col(meta, "ads_category")
    meta["ads_reward_price"]   = _to_num(meta["ads_reward_price"])
    meta["ads_contract_price"] = _to_num(meta["ads_contract_price"])
    meta["price_reward_log"]   = np.log1p(meta["ads_reward_price"])
    meta["price_ratio"]        = (meta["ads_reward_price"] /
                                  meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)
    # (단일 광고 메타만 쓰면 되므로 첫 행만 사용)
    meta_row = meta.iloc[[0]][["ads_idx","ads_category","price_reward_log","price_ratio"]].copy()

    # 4) 그리드: ad × (활성)매체 cross join
    grid = meta_row.merge(mda_active, how="cross")

    # 5) 카테고리 정규화 후 cat_cvr 조인
    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr"]].copy(), "ads_category")
    grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0})

    # 6) 스코어링 → TOP-K
    X = ensure_feature_matrix(grid, feat_cols)
    scores = ranker.predict(X)
    grid = grid[["ads_idx","mda_idx","mda_cvr_global","mda_pop_norm","cat_cvr"]].copy()
    grid["score"] = scores

    top = grid.nlargest(min(topk, len(grid)), "score").reset_index(drop=True)
    return top


In [ ]:
pred_table = search_and_predict_ad('157236', obs, ads_list, ranker, feat_cols, topk=10)
print(pred_table)


   ads_idx  mda_idx  mda_cvr_global  mda_pop_norm   cat_cvr     score
0   157236      563        0.573331      8.783843  0.598385  5.652098
1   157236      562        0.584824      0.121230  0.593577  1.077826
2   157236      371        0.419756      0.050384  0.435572  0.375843
3   157236      270        0.485917      0.662206  0.548246 -0.087617
4   157236      634        0.616186     -0.171478  0.644345 -1.415706
5   157236      761        0.177646      0.134623  0.226987 -2.127146
6   157236       54        0.349781     -0.121672  0.263158 -2.392473
7   157236      442        0.374150     -0.268464  0.500000 -2.565647
8   157236      817        0.700431     -0.245825  0.405941 -3.124384
9   157236      389        0.659574     -0.251685  0.300000 -3.295231


# 지표

In [ ]:
def evaluate_all(preds_dict, gt_dict, k=10):
    """
    preds_dict: {ad: [(mda_idx, score), ...]}
    gt_dict: {ad: [mda_idx1, mda_idx2, ...]}
    """
    P, R, H, APs, DCGs, IDCGs = [], [], [], [], [], []
    for ad, lst in preds_dict.items():
        if ad not in gt_dict: 
            continue
        pred = [m for m,_ in lst[:k]]
        true = set(gt_dict[ad])
        if not true: 
            continue
        
        # Precision / Recall / Hit
        inter = len(set(pred) & true)
        P.append(inter / max(1, len(pred)))
        R.append(inter / len(true))
        H.append(1.0 if inter > 0 else 0.0)
        
        # AP@K
        hits, sum_prec = 0, 0.0
        for i, m in enumerate(pred, start=1):
            if m in true:
                hits += 1
                sum_prec += hits / i
        APs.append(sum_prec / max(1, min(len(true), k)))
        
        # NDCG@K
        y_true = np.zeros(len(lst))
        y_score = np.zeros(len(lst))
        mda_ids = [m for m,_ in lst]
        for j, m in enumerate(mda_ids):
            if m in true:
                y_true[j] = 1
            y_score[j] = lst[j][1]
        # sklearn의 ndcg_score은 2D array 필요
        ndcg = ndcg_score([y_true], [y_score], k=k)
        DCGs.append(ndcg)
    
    return {
        f"precision@{k}": float(np.mean(P)) if P else 0.0,
        f"recall@{k}":    float(np.mean(R)) if R else 0.0,
        f"hit_rate@{k}":  float(np.mean(H)) if H else 0.0,
        f"map@{k}":       float(np.mean(APs)) if APs else 0.0,
        f"ndcg@{k}":      float(np.mean(DCGs)) if DCGs else 0.0,
        "evaluated_ads":  len(P)
    }


In [ ]:
gt = tgt.groupby("ads_idx")["mda_idx"].apply(list).to_dict()
results = evaluate_all(preds, gt, k=5)
print(results)


{'precision@5': 0.20666666666666667, 'recall@5': 0.46708150871684706, 'hit_rate@5': 0.6, 'map@5': 0.41844444444444445, 'ndcg@5': 0.4664113062713906, 'evaluated_ads': 30}


In [ ]:
# -------------------------------
# Train/Valid/Test Split
# -------------------------------
def split_train_valid_test(df: pd.DataFrame,
                           test_ads_recent: int = 20,
                           valid_ratio: float = 0.2,
                           rng_seed: int = 42):
    """
    학습/검증/테스트 분할
    - 가장 최근 광고 n개를 테스트로
    - 나머지 광고 중 일정 비율을 검증(valid)으로 분리
    - 도메인/카테고리 편중 방지를 위해 stratified 방식 사용 가능
    """

    # (1) ads별 마지막 집행 날짜 계산
    ad_last_date = (df.groupby("ads_idx")["tgt_turn"]
                      .sum()
                      .reset_index())  # 단순히 sum 기준으로도 가능
    # 실제라면 rpt_time_date의 max 기준으로 정렬하는 게 더 정확
    # ad_last_date = df.groupby("ads_idx")["date"].max().reset_index()

    # (2) 최근 광고 N개 = Test
    recent_ads = ad_last_date.sort_values("ads_idx", ascending=False) \
                             .head(test_ads_recent)["ads_idx"].tolist()
    test_mask = df["ads_idx"].isin(recent_ads)

    df_test = df[test_mask].copy()
    df_rest = df[~test_mask].copy()

    # (3) 나머지 → Train/Valid Split
    ads_all = df_rest["ads_idx"].unique()
    rng = np.random.RandomState(rng_seed)
    rng.shuffle(ads_all)

    n_valid = int(len(ads_all) * valid_ratio)
    valid_ads = set(ads_all[:n_valid])

    df_valid = df_rest[df_rest["ads_idx"].isin(valid_ads)].copy()
    df_train = df_rest[~df_rest["ads_idx"].isin(valid_ads)].copy()

    print(f"[Split] Train={len(df_train)} rows | Valid={len(df_valid)} rows | Test={len(df_test)} rows")
    print(f"        ads (Train={df_train['ads_idx'].nunique()}, "
          f"Valid={df_valid['ads_idx'].nunique()}, "
          f"Test={df_test['ads_idx'].nunique()})")

    return df_train, df_valid, df_test, recent_ads


In [ ]:
# ==============================================
#  Ad × Media 추천 (Attribute-only, Cold-start Safe, Fast Inference)
#  Precision 향상 버전: 후보 보수화 + 하드네거 다운샘플 + label_gain 강화 + 추가 피처(cat_rel, mda_conv_q)
# ==============================================

warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 공통 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str = "ads_category") -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s):
    return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

# -------------------------------
# 1) 윈도우 (최신 달, 마지막 7일 TGT)
# -------------------------------
def split_obs_tgt_month(ads_time: pd.DataFrame, tgt_days: int = 7):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    last_date = t["date"].max()
    if pd.isna(last_date):
        raise ValueError("ads_time['rpt_time_date']에서 유효한 날짜를 찾을 수 없습니다.")
    month_start = last_date.normalize().replace(day=1)
    next_month  = (month_start + pd.offsets.MonthBegin(1))
    month_end   = (next_month - pd.Timedelta(days=1)).normalize()
    cutoff      = month_end - pd.Timedelta(days=tgt_days)
    obs = t[(t["date"] >= month_start) & (t["date"] <= cutoff)].copy()
    tgt = t[(t["date"] >  cutoff) & (t["date"] <= month_end)].copy()
    return obs, tgt, month_start, month_end, cutoff

# -------------------------------
# 2) 속성 컨텍스트(OBS만 사용)
# -------------------------------
def build_attr_context(obs: pd.DataFrame, ads_list: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # 매체 전반 (OBS)
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"),
                      mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0, np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std()
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 매체 전환수 분위수(강한 인기 가중에 활용)
    mda_all["mda_conv_q"] = pd.qcut(
        mda_all["mda_conv_global"].rank(method="first"),
        q=10, labels=False, duplicates="drop"
    ).astype(float)

    # 카테고리×매체 (OBS)
    al = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])

    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"] / cat_mda["clk"].replace(0, np.nan)).fillna(0.0)

    return {"mda_all": mda_all, "cat_mda": cat_mda}

# -------------------------------
# 3) 학습 대상 광고 선별
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000,
                            rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1, int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성 (Precision 지향 보수화)
# -------------------------------
def create_candidates(
    ads_list: pd.DataFrame,
    mda_all: pd.DataFrame,
    cat_mda: pd.DataFrame,
    topC: int = 5, add_random: int = 1, cat_topM: int = 3,
    min_active_clk: int = 300, min_active_conv: int = 30,
    rng_seed: int = 42,
    ad_ids_filter: Optional[np.ndarray] = None,
    max_per_ad: int = 10
) -> pd.DataFrame:
    mda_active = mda_all[
        (mda_all["mda_clk_global"]  >= min_active_clk) |
        (mda_all["mda_conv_global"] >= min_active_conv)
    ].copy()
    if mda_active.empty:
        mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)

    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True, False])
                      .groupby("ads_category")
                      .head(cat_topM)[["ads_category","mda_idx"]].copy())
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c, g in cat_top.groupby("ads_category")}

    rng = np.random.RandomState(rng_seed)
    if ad_ids_filter is not None and len(ad_ids_filter) > 0:
        ad_ids = np.array(ad_ids_filter, dtype=int)
    else:
        ad_ids = ads_list["ads_idx"].dropna().astype(int).unique()

    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs = []
    for ad in ad_ids:
        base = []
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC, len(mda_ids_global))])
        base = list(dict.fromkeys(base))
        remain = np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain) > 0 and add_random > 0:
            base.extend(rng.choice(remain, size=min(add_random, len(remain)), replace=False).tolist())
        if len(base) > max_per_ad: base = base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]

    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} ads={len(ad_ids):,} active_mda={len(mda_active):,}")
    return out

# -------------------------------
# 5) 학습 데이터 준비 (속성 전용)
# -------------------------------
FEAT_ALLOW = [
    "mda_cvr_global","mda_pop_norm","cat_cvr",
    "price_reward_log","price_ratio",
    "cat_rel","mda_conv_q"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list: pd.DataFrame,
    **cand_kwargs
):
    train_ads = pick_train_ads_from_tgt(tgt)
    actx = build_attr_context(obs, ads_list)
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]

    cand = create_candidates(
        ads_list=ads_list,
        mda_all=mda_all, cat_mda=cat_mda,
        ad_ids_filter=train_ads, **cand_kwargs
    )

    df = cand.merge(mda_all[["mda_idx","mda_cvr_global","mda_pop_norm",
                             "mda_conv_q","mda_clk_global","mda_conv_global"]],
                    on="mda_idx", how="left")

    ad_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]] \
                .drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"] = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_ratio"]      = (ad_meta["ads_reward_price"] /
                                   ad_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","price_reward_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0})

    # 추가 피처
    df["cat_rel"] = df["cat_cvr"] / (df["mda_cvr_global"] + 1e-9)

    # 라벨
    tgt_g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
               .agg(tgt_turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    df = df.merge(tgt_g, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})

    # relevance bin
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum() > 0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int) + 1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 피처 매트릭스 / 다운샘플 / 학습
# -------------------------------
LABEL_GAIN = [0.0, 1.0, 6.0, 24.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c] = 0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad_hard(df, ratio=4, hard_top=30, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"] > 0]; neg = g[g["tgt_turn"] <= 0]
        if len(pos) == 0: continue
        hard = (neg.sort_values(["cat_cvr","mda_cvr_global"], ascending=False)
                    .head(hard_top))
        remain = neg.drop(hard.index)
        cap = len(pos) * ratio - len(hard)
        if cap > 0 and len(remain) > 0:
            rand = remain.sample(n=min(cap, len(remain)), random_state=rng)
            neg_keep = pd.concat([hard, rand], ignore_index=True)
        else:
            neg_keep = hard
        out.append(pd.concat([pos, neg_keep], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    w = np.ones(len(train_df), dtype=float)
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10],
        learning_rate=0.03, n_estimators=4000, num_leaves=31,
        min_data_in_leaf=60, lambda_l2=5.0, subsample=0.9, colsample_bytree=0.9,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=w)
    return ranker, feat_cols

# -------------------------------
# 7) 예측 (per-ad 후보 + GT 강제 포함) — 오프라인 평가용
# -------------------------------
def predict_with_per_ad_candidates_for_eval(
    ranker, feat_cols, actx, ads_list, tgt, K=10,
    topC=5, cat_topM=3, add_random=1,
    min_active_clk=300, min_active_conv=30
) -> Dict[int, List[Tuple[int,float]]]:
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
    active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                     (mda_all["mda_conv_global"]>=min_active_conv)]
    if active.empty: active = mda_all.copy()
    mda_ids_global = active.sort_values("mda_conv_global", ascending=False)["mda_idx"].astype(int).tolist()

    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True, False])
                            .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}

    ads_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx").copy()
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] /
                                      ads_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)
    ad2row = ads_meta.set_index("ads_idx")[["ads_category","price_reward_log","price_ratio"]].to_dict("index")

    eval_ads = pd.to_numeric(tgt["ads_idx"], errors="coerce").dropna().astype(int).unique().tolist()
    gt_map = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: pd.to_numeric(s, errors="coerce").dropna().astype(int).tolist()).to_dict()

    cat_mda_local = _norm_cat_col(actx["cat_mda"][["mda_idx","ads_category","cat_cvr"]].copy(), "ads_category")
    preds = {}
    rng = np.random.RandomState(42)

    for ad in eval_ads:
        meta = ad2row.get(int(ad), {"ads_category":"__na__","price_reward_log":0.0,"price_ratio":0.0})
        cat  = meta["ads_category"]

        base = []
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC, len(mda_ids_global))])
        base = list(dict.fromkeys(base))
        remain = np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random, len(remain)), replace=False).tolist())

        base = list(dict.fromkeys(base + gt_map.get(ad, [])))
        if not base: preds[ad] = []; continue

        g = pd.DataFrame({"ads_idx": ad, "mda_idx": base})
        g = g.merge(mda_all[["mda_idx","mda_cvr_global","mda_pop_norm","mda_conv_q"]], on="mda_idx", how="left")
        g["ads_category"]     = cat
        g["price_reward_log"] = meta["price_reward_log"]
        g["price_ratio"]      = meta["price_ratio"]
        g = g.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0})
        g["cat_rel"] = g["cat_cvr"] / (g["mda_cvr_global"] + 1e-9)

        X = ensure_feature_matrix(g, feat_cols)          # ✅ 오타 수정됨
        s = ranker.predict(X)
        top = np.argsort(-s)[:min(K, len(s))]
        preds[ad] = list(zip(g.iloc[top]["mda_idx"].astype(int).tolist(),
                             s[top].astype(float).tolist()))
    return preds

# 운영용: 특정 광고 리스트만 빠르게 예측
def predict_for_specific_ads_fast(
    ranker, feat_cols, actx, ads_list, ad_id_list, K=10,
    topC=5, cat_topM=3, add_random=1,
    min_active_clk=300, min_active_conv=30
):
    ad_id_list = pd.Series(pd.to_numeric(ad_id_list, errors="coerce")).dropna().astype(int)
    exist_ads = set(pd.to_numeric(ads_list["ads_idx"], errors="coerce").dropna().astype(int))
    ad_id_list = [a for a in ad_id_list if a in exist_ads]
    if not ad_id_list:
        return {}
    tgt_dummy = pd.DataFrame({"ads_idx": ad_id_list})
    tgt_dummy["mda_idx"] = pd.Series(dtype=int)
    return predict_with_per_ad_candidates_for_eval(
        ranker, feat_cols, actx, ads_list, tgt_dummy, K=K,
        topC=topC, cat_topM=cat_topM, add_random=add_random,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv
    )

# -------------------------------
# 8) GT 구성/지표 (Precision/Recall/Hit/MAP/NDCG)
# -------------------------------
def make_gt_sets_and_gains(tgt: pd.DataFrame):
    g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
           .agg(turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    gt_gain = {int(ad): {int(r["mda_idx"]): float(r["turn"]) for _, r in gg.iterrows()}
               for ad, gg in g.groupby("ads_idx")}
    gt_set  = {ad: set(d.keys()) for ad, d in gt_gain.items()}
    return gt_set, gt_gain

def _precision_at_k(gt_set, pred_list, k):
    if k <= 0: return np.nan
    k = min(k, len(pred_list))
    return sum(1 for a in pred_list[:k] if a in gt_set) / max(1, k)

def _recall_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    return sum(1 for a in pred_list[:k] if a in gt_set) / len(gt_set)

def _map_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    hits, ap = 0, 0.0
    for i, a in enumerate(pred_list[:k], start=1):
        if a in gt_set:
            hits += 1; ap += hits / i
    return ap / min(len(gt_set), k)

def _ndcg_at_k_from_gains(gains_in_pred_order, k):
    k = min(k, len(gains_in_pred_order))
    dcg  = sum(g / math.log2(i+2) for i, g in enumerate(gains_in_pred_order[:k]))
    ideal_gains = sorted(gains_in_pred_order, reverse=True)[:k]
    idcg = sum(g / math.log2(i+2) for i, g in enumerate(ideal_gains))
    return dcg / idcg if idcg > 0 else 0.0

def _ndcg_at_k(gt_gain_map, pred_list, k):
    gains = [gt_gain_map.get(a, 0.0) for a in pred_list[:k]]
    return _ndcg_at_k_from_gains(gains, k)

def evaluate_with_gain(preds, gt_set, gt_gain, k=10):
    rows = []
    for ad, lst in preds.items():
        if ad not in gt_set: continue
        pred_mdas = [m for m,_ in lst]
        row = {
            "P": _precision_at_k(gt_set[ad], pred_mdas, k),
            "R": _recall_at_k(gt_set[ad], pred_mdas, k),
            "MAP": _map_at_k(gt_set[ad], pred_mdas, k),
            "NDCG": _ndcg_at_k(gt_gain.get(ad, {}), pred_mdas, k),
            "Hit": 1.0 if any(m in gt_set[ad] for m in pred_mdas[:k]) else 0.0
        }
        rows.append(row)
    if not rows:
        return {f"precision@{k}":0.0, f"recall@{k}":0.0,
                f"hit_rate@{k}":0.0, f"map@{k}":0.0, f"ndcg@{k}":0.0,
                "evaluated_ads":0}
    df = pd.DataFrame(rows)
    return {
        f"precision@{k}": float(df["P"].mean()),
        f"recall@{k}":    float(df["R"].mean()),
        f"hit_rate@{k}":  float(df["Hit"].mean()),
        f"map@{k}":       float(df["MAP"].mean()),
        f"ndcg@{k}":      float(df["NDCG"].mean()),
        "evaluated_ads":  int(len(df))
    }

def summarize_metrics(metrics_by_k: Dict[int, Dict[str, float]]) -> pd.DataFrame:
    rows = []
    for k, m in sorted(metrics_by_k.items()):
        rows.append([k, m.get(f"precision@{k}",0.0), m.get(f"recall@{k}",0.0),
                     m.get(f"hit_rate@{k}",0.0), m.get(f"map@{k}",0.0),
                     m.get(f"ndcg@{k}",0.0), m.get("evaluated_ads",0)])
    return pd.DataFrame(rows, columns=["K","Precision","Recall","HitRate","MAP","NDCG","#ads"])

# ============================================================
# ==============          사용 예시           =================
# ============================================================
# 준비물: ads_time, ads_list (pandas DataFrame)

# 1) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_month(ads_time, tgt_days=7)
print(f"[Window] OBS:{start.date()}→{cut.date()} | TGT:{(cut+pd.Timedelta(days=1)).date()}→{end.date()}")

# 2) 학습 데이터 (보수 후보 파라미터)
train_df, actx = prepare_training_attr_only(
    obs, tgt, ads_list,
    topC=5, add_random=1, cat_topM=3,
    min_active_clk=300, min_active_conv=30,
    max_per_ad=10  # 전달해도 무시되지 않게 create_candidates에 이미 있음
)
print("train_df size:", len(train_df))

# 3) 다운샘플(하드네거 우선) + 학습
train_df = downsample_negatives_per_ad_hard(train_df, ratio=4, hard_top=30)
ranker, feat_cols = train_attr_ranker(train_df)
print("feat_cols used:", feat_cols)

# 4) 오프라인 평가 (TGT 광고 대상으로 per-ad 후보 + GT 강제포함)
preds = predict_with_per_ad_candidates_for_eval(
    ranker, feat_cols, actx, ads_list, tgt, K=10,
    topC=5, cat_topM=3, add_random=1,
    min_active_clk=300, min_active_conv=30
)
gt_set, gt_gain = make_gt_sets_and_gains(tgt)
metrics = {k: evaluate_with_gain(preds, gt_set, gt_gain, k=k) for k in [5,10,20]}
print(metrics)
print(summarize_metrics(metrics))

# 5) (운영/조회) 특정 광고만 빠르게 예측
ad_list = [157236, 12345, 67890]
preds_small = predict_for_specific_ads_fast(
    ranker, feat_cols, actx, ads_list, ad_list, K=10,
    topC=5, cat_topM=3, add_random=1
)
print(preds_small.get(157236, [])[:10])


[Window] OBS:2025-08-01→2025-08-24 | TGT:2025-08-25→2025-08-31
[Candidates] pairs=5,665 ads=637 active_mda=137
train_df size: 5665
feat_cols used: ['mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'price_reward_log', 'price_ratio', 'cat_rel', 'mda_conv_q']
{5: {'precision@5': 0.2786697247706422, 'recall@5': 0.9379148177045138, 'hit_rate@5': 0.9908256880733946, 'map@5': 0.800493119266055, 'ndcg@5': 0.6624559170181598, 'evaluated_ads': 872}, 10: {'precision@10': 0.16880005824959957, 'recall@10': 0.9826829280134669, 'hit_rate@10': 1.0, 'map@10': 0.8115575250551291, 'ndcg@10': 0.6607852507253086, 'evaluated_ads': 872}, 20: {'precision@20': 0.16880005824959957, 'recall@20': 0.9826829280134669, 'hit_rate@20': 1.0, 'map@20': 0.8040445228405938, 'ndcg@20': 0.6607852507253086, 'evaluated_ads': 872}}
    K  Precision    Recall   HitRate       MAP      NDCG  #ads
0   5    0.27867  0.937915  0.990826  0.800493  0.662456   872
1  10    0.16880  0.982683  1.000000  0.811558  0.660785   872
2  20    0.16

In [ ]:
# ==============================================
# Ad × Media 추천 (Attribute-only, Cold-start Safe)
# ==============================================

warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")
def _norm_cat_col(df: pd.DataFrame, col: str="ads_category") -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__",
                                   "none":"__na__", "nat":"__na__"})
    return df

# -------------------------------
# 1) 윈도우
# -------------------------------
def split_obs_tgt_month(ads_time: pd.DataFrame, tgt_days: int=7):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    last_date = t["date"].max()
    month_start = last_date.normalize().replace(day=1)
    next_month  = month_start + pd.offsets.MonthBegin(1)
    month_end   = (next_month - pd.Timedelta(days=1)).normalize()
    cutoff      = month_end - pd.Timedelta(days=tgt_days)
    obs = t[(t["date"] >= month_start) & (t["date"] <= cutoff)].copy()
    tgt = t[(t["date"] > cutoff) & (t["date"] <= month_end)].copy()
    return obs, tgt, month_start, month_end, cutoff

# -------------------------------
# 2) 속성 컨텍스트
# -------------------------------
def build_attr_context(obs, ads_list):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"),
                      mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std()
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"]-mu)/(sd+1e-9)

    al = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = al["ads_category"].astype(str).str.strip().str.lower()
    t2 = t.merge(al, on="ads_idx", how="left")

    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    return {"mda_all": mda_all, "cat_mda": cat_mda}

# -------------------------------
# 3) 후보 생성
# -------------------------------
def create_candidates(ads_list, mda_all, cat_mda,
                      topC=10, cat_topM=5, add_random=2,
                      min_active_clk=200, min_active_conv=20,
                      max_per_ad=20, ad_ids_filter=None):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)]
    if mda_active.empty: mda_active=mda_all
    mda_ids_global = mda_active.sort_values("mda_conv_global",ascending=False)["mda_idx"].astype(int).values

    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"],ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c:g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}

    if ad_ids_filter is not None:
        ad_ids = np.array(ad_ids_filter,dtype=int)
    else:
        ad_ids = ads_list["ads_idx"].dropna().astype(int).unique()

    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat=ad2cat.get(int(ad),None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:topC])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global,np.array(base,dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(np.random.choice(remain,size=min(add_random,len(remain)),replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs+=[(int(ad),int(m)) for m in base]

    out=pd.DataFrame(pairs,columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} ads={len(ad_ids):,} active_mda={len(mda_active):,}")
    return out

# -------------------------------
# 4) 학습 데이터 준비
# -------------------------------
FEAT_ALLOW=["mda_cvr_global","mda_pop_norm","cat_cvr","price_reward_log","price_ratio"]

def prepare_training_attr_only(obs,tgt,ads_list):
    actx=build_attr_context(obs,ads_list)
    mda_all,cat_mda=actx["mda_all"],actx["cat_mda"]

    cand=create_candidates(ads_list,mda_all,cat_mda)
    df=cand.merge(mda_all[["mda_idx","mda_cvr_global","mda_pop_norm"]],on="mda_idx",how="left")

    ad_meta=ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx").copy()
    ad_meta=_norm_cat_col(ad_meta,"ads_category")
    for c in ["ads_reward_price","ads_contract_price"]: ad_meta[c]=_to_num(ad_meta[c])
    ad_meta["price_reward_log"]=np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_ratio"]=(ad_meta["ads_reward_price"]/
                            ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df=df.merge(ad_meta[["ads_idx","ads_category","price_reward_log","price_ratio"]],on="ads_idx",how="left")
    df=_norm_cat_col(df,"ads_category"); cat_mda=_norm_cat_col(cat_mda,"ads_category")
    df=df.merge(cat_mda[["mda_idx","ads_category","cat_cvr"]],
                on=["mda_idx","ads_category"],how="left").fillna({"cat_cvr":0.0})

    tgt_g=(tgt.groupby(["ads_idx","mda_idx"],as_index=False)
             .agg(tgt_turn=("rpt_time_turn",lambda s:_to_num(s).sum())))
    df=df.merge(tgt_g,on=["ads_idx","mda_idx"],how="left").fillna({"tgt_turn":0.0})

    y=df["tgt_turn"].astype(float); rel=np.zeros(len(df),dtype=int); pos=y>0
    if pos.sum()>0:
        ylog=np.log1p(y[pos])
        try:
            bins=pd.qcut(ylog,q=3,labels=False,duplicates="drop"); rel[pos]=bins.astype(int)+1
        except: rel[pos]=1
    df["relevance"]=rel
    return df,actx

# -------------------------------
# 5) 학습
# -------------------------------
def ensure_feature_matrix(df,feat_cols):
    X=df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype,np.number): X[c]=pd.to_numeric(X[c],errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df,ratio=4):
    out=[]
    for ad,g in df.groupby("ads_idx",sort=False):
        pos=g[g["tgt_turn"]>0]; neg=g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap=len(pos)*ratio
        if len(neg)>cap: neg=neg.sample(n=cap,random_state=42)
        out.append(pd.concat([pos,neg],ignore_index=True))
    return pd.concat(out,ignore_index=True) if out else df

def train_attr_ranker(train_df):
    feat_cols=[c for c in FEAT_ALLOW if c in train_df.columns]
    X=ensure_feature_matrix(train_df,feat_cols)
    y=train_df["relevance"].astype(int)
    groups=train_df.groupby("ads_idx").size().astype(int).tolist()
    w=np.ones(len(train_df))
    ranker=lgb.LGBMRanker(objective="lambdarank",metric="ndcg",eval_at=[5,10,20],
                          learning_rate=0.05,n_estimators=1200,num_leaves=47,
                          min_data_in_leaf=40,lambda_l2=1.5,subsample=0.85,
                          colsample_bytree=0.85,label_gain=[0,1,4,12],
                          random_state=42,n_jobs=-1)
    ranker.fit(X,y,group=groups,sample_weight=w)
    return ranker,feat_cols

# -------------------------------
# 6) 평가
# -------------------------------
def evaluate_all(preds_dict, gt_dict, ks=[5,10,20]):
    results={}
    for k in ks:
        P,R,H,APs,Ns=[],[],[],[],[]
        for ad,lst in preds_dict.items():
            if ad not in gt_dict: continue
            pred=[m for m,_ in lst[:k]]; true=set(gt_dict[ad])
            if not true: continue
            inter=set(pred)&true
            P.append(len(inter)/max(1,len(pred)))
            R.append(len(inter)/len(true))
            H.append(1.0 if inter else 0.0)
            # MAP@k
            ap=0.0; hit=0
            for i,m in enumerate(pred,1):
                if m in true: hit+=1; ap+=hit/i
            APs.append(ap/max(1,len(true)))
            # nDCG@k
            dcg=0.0; idcg=sum(1.0/np.log2(i+2) for i in range(min(len(true),k)))
            for i,m in enumerate(pred):
                if m in true: dcg+=1.0/np.log2(i+2)
            Ns.append(dcg/idcg if idcg>0 else 0.0)
        results[k]={"precision@%d"%k:np.mean(P) if P else 0.0,
                    "recall@%d"%k:np.mean(R) if R else 0.0,
                    "hit_rate@%d"%k:np.mean(H) if H else 0.0,
                    "map@%d"%k:np.mean(APs) if APs else 0.0,
                    "ndcg@%d"%k:np.mean(Ns) if Ns else 0.0,
                    "evaluated_ads":len(P)}
    return results

# -------------------------------
# 메인 실행
# -------------------------------
obs,tgt,start,end,cut=split_obs_tgt_month(ads_time,tgt_days=7)
print(f"[Window] OBS:{start.date()}→{cut.date()} | TGT:{(cut+pd.Timedelta(days=1)).date()}→{end.date()}")

train_df,actx=prepare_training_attr_only(obs,tgt,ads_list)
print("train_df size:",len(train_df))
train_df=downsample_negatives_per_ad(train_df,ratio=4)
ranker,feat_cols=train_attr_ranker(train_df)
print("feat_cols used:",feat_cols)

# 예측
gt=tgt.groupby("ads_idx")["mda_idx"].apply(list).to_dict()
# 배치 예측 대신 필요한 광고만 빠르게 조회하는 함수도 작성 가능


[Window] OBS:2025-08-01→2025-08-24 | TGT:2025-08-25→2025-08-31
[Candidates] pairs=7,135,452 ads=445,039 active_mda=139
train_df size: 7135452
feat_cols used: ['mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'price_reward_log', 'price_ratio']


In [ ]:
# ==============================================
#  Ad × Media 추천 (Attribute-only, Cold-start Safe, Eval/Serve One-Click)
#  - 피처: mda_cvr_global, mda_pop_norm, cat_cvr, price_reward_log, price_ratio (+ 안전 파생)
#  - 모드:
#     * eval  : 후보 넓게, K=[5,10,20] 성능 리포트
#     * serve : 후보 좁게, 특정 광고 리스트만 빠르게 Top-K
# ==============================================

from __future__ import annotations
import math, warnings
from typing import Optional, Iterable, Dict, List, Tuple
import numpy as np
import pandas as pd
import lightgbm as lgb

warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 공통 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str = "ads_category") -> pd.DataFrame:
    """컬럼을 문자열로 통일(merge dtype 오류 방지)"""
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s):
    """Series를 문자열/소문자/trim으로 표준화"""
    return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

# -------------------------------
# 1) 윈도우 (최신 달, 마지막 7일 TGT)
# -------------------------------
def split_obs_tgt_month(ads_time: pd.DataFrame, tgt_days: int = 7):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    last_date = t["date"].max()
    if pd.isna(last_date):
        raise ValueError("ads_time['rpt_time_date']에서 유효한 날짜를 찾을 수 없습니다.")
    month_start = last_date.normalize().replace(day=1)
    next_month  = (month_start + pd.offsets.MonthBegin(1))
    month_end   = (next_month - pd.Timedelta(days=1)).normalize()
    cutoff      = month_end - pd.Timedelta(days=tgt_days)
    obs = t[(t["date"] >= month_start) & (t["date"] <= cutoff)].copy()
    tgt = t[(t["date"] >  cutoff) & (t["date"] <= month_end)].copy()
    return obs, tgt, month_start, month_end, cutoff

# -------------------------------
# 2) 속성 컨텍스트(OBS만 사용)
# -------------------------------
def build_attr_context(obs: pd.DataFrame, ads_list: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # 매체 전반 (OBS)
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"),
                      mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0, np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std()
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체 (OBS)
    al = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])

    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"] / cat_mda["clk"].replace(0, np.nan)).fillna(0.0)

    return {"mda_all": mda_all, "cat_mda": cat_mda}

# -------------------------------
# 3) 학습 대상 광고 선별 (TGT에서 양성 존재 + 상한)
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000,
                            rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1, int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성(속성 기반, 강력 축소)
# -------------------------------
def create_candidates(
    ads_list: pd.DataFrame,
    mda_all: pd.DataFrame,
    cat_mda: pd.DataFrame,
    topC: int = 10, add_random: int = 2, cat_topM: int = 5,
    min_active_clk: int = 200, min_active_conv: int = 20,
    rng_seed: int = 42,
    ad_ids_filter: Optional[np.ndarray] = None,
    max_per_ad: int = 20
) -> pd.DataFrame:
    # 비활성 매체 컷
    mda_active = mda_all[
        (mda_all["mda_clk_global"]  >= min_active_clk) |
        (mda_all["mda_conv_global"] >= min_active_conv)
    ].copy()
    if mda_active.empty:
        mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)

    # 카테고리 상위 캐시
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True, False])
                      .groupby("ads_category")
                      .head(cat_topM)[["ads_category","mda_idx"]].copy())
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c, g in cat_top.groupby("ads_category")}

    rng = np.random.RandomState(rng_seed)
    # 광고 대상 제한
    if ad_ids_filter is not None and len(ad_ids_filter) > 0:
        ad_ids = np.array(ad_ids_filter, dtype=int)
    else:
        ad_ids = ads_list["ads_idx"].dropna().astype(int).unique()

    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs = []
    for ad in ad_ids:
        base = []
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top:
            base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC, len(mda_ids_global))])
        base = list(dict.fromkeys(base))
        # 랜덤
        remain = np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain) > 0 and add_random > 0:
            base.extend(rng.choice(remain, size=min(add_random, len(remain)), replace=False).tolist())
        # ad별 최대 후보 수 제한
        if len(base) > max_per_ad:
            base = base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]

    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    return out

# -------------------------------
# 5) 학습 데이터(허용 피처만 생성)
# -------------------------------
FEAT_ALLOW = [
    "mda_cvr_global",   # 매체 전환율(OBS)
    "mda_pop_norm",     # 매체 전환수 기반 표준화 인기(OBS)
    "cat_cvr",          # 카테고리×매체 CVR(OBS)
    "price_reward_log", # log(보상가)
    "price_ratio"       # 보상/계약 비율
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    max_per_ad: int = 20
):
    # 학습 대상 광고 제한
    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt,
                                        max_train_ads=max_train_ads)

    actx = build_attr_context(obs, ads_list)
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]

    # 후보 생성(축소)
    cand = create_candidates(
        ads_list=ads_list,
        mda_all=mda_all,
        cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=max_per_ad
    )
    print(f"[Candidates] pairs={len(cand):,} ads={len(np.unique(cand['ads_idx'])):,} active_mda={len(mda_all)}")

    # 매체 피처(전역 허용분)
    df = cand.merge(mda_all[["mda_idx","mda_cvr_global","mda_pop_norm",
                             "mda_clk_global","mda_conv_global"]],
                    on="mda_idx", how="left")

    # 광고 피처
    ad_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]] \
                .drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"] = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_ratio"]      = (ad_meta["ads_reward_price"] /
                                   ad_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","price_reward_log","price_ratio"]],
                  on="ads_idx", how="left")

    # 카테고리×매체 적합도
    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0})

    # 라벨(TGT 전환수) — 피처엔 사용하지 않음
    tgt_g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
               .agg(tgt_turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    df = df.merge(tgt_g, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})

    # relevance bin (0=neg, 1/2/3=pos bins)
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum() > 0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int) + 1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 피처 매트릭스 / 다운샘플 / 학습
# -------------------------------
LABEL_GAIN = [0.0, 1.0, 4.0, 12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c] = 0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"] > 0]; neg = g[g["tgt_turn"] <= 0]
        if len(pos) == 0: continue
        cap = len(pos) * ratio
        if len(neg) > cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos, neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    w = np.ones(len(train_df), dtype=float)
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
        learning_rate=0.05, n_estimators=1200, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=w)
    return ranker, feat_cols

# -------------------------------
# 7) 예측(빠른 버전) — 평가용 per-ad 후보 + GT 강제 포함
# -------------------------------
def predict_with_per_ad_candidates_for_eval(
    ranker, feat_cols, actx, ads_list, tgt, K=20,
    topC=10, cat_topM=5, add_random=2,
    min_active_clk=200, min_active_conv=20
) -> Dict[int, List[Tuple[int,float]]]:
    mda_all, cat_mda = actx["mda_all"], actx["cat_mda"]
    # 활성 매체
    active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                     (mda_all["mda_conv_global"]>=min_active_conv)]
    if active.empty: active = mda_all.copy()
    mda_ids_global = active.sort_values("mda_conv_global", ascending=False)["mda_idx"].astype(int).tolist()

    # 카테고리 상위 캐시
    cat_top = (actx["cat_mda"].sort_values(["ads_category","cat_cvr"], ascending=[True, False])
                            .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}

    # 광고 메타
    ads_meta = ads_list[["ads_idx","ads_category","ads_reward_price","ads_contract_price"]].drop_duplicates("ads_idx").copy()
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] /
                                      ads_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)
    ad2row = ads_meta.set_index("ads_idx")[["ads_category","price_reward_log","price_ratio"]].to_dict("index")

    # 평가 대상 광고(= TGT에 있는 광고)
    eval_ads = pd.to_numeric(tgt["ads_idx"], errors="coerce").dropna().astype(int).unique().tolist()
    # 광고별 GT 매체
    gt_map = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: pd.to_numeric(s, errors="coerce").dropna().astype(int).tolist()).to_dict()

    cat_mda_local = _norm_cat_col(actx["cat_mda"][["mda_idx","ads_category","cat_cvr"]].copy(), "ads_category")
    preds = {}
    rng = np.random.RandomState(42)

    for ad in eval_ads:
        meta = ad2row.get(int(ad), {"ads_category":"__na__", "price_reward_log":0.0, "price_ratio":0.0})
        cat  = meta["ads_category"]

        # per-ad 후보: 카테고리 상위 + 글로벌 상위 + 소량 랜덤
        base = []
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC, len(mda_ids_global))])
        base = list(dict.fromkeys(base))
        remain = np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random, len(remain)), replace=False).tolist())

        # GT 매체 강제 포함
        base = list(dict.fromkeys(base + gt_map.get(ad, [])))
        if not base:
            preds[ad] = []
            continue

        # 그리드 & 점수
        g = pd.DataFrame({"ads_idx": ad, "mda_idx": base})
        g = g.merge(mda_all[["mda_idx","mda_cvr_global","mda_pop_norm"]], on="mda_idx", how="left")
        g["ads_category"]     = cat
        g["price_reward_log"] = meta["price_reward_log"]
        g["price_ratio"]      = meta["price_ratio"]
        g = g.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0})

        X = ensure_feature_matrix(g, feat_cols)
        s = ranker.predict(X)
        top = np.argsort(-s)[:min(K, len(s))]
        preds[ad] = list(zip(g.iloc[top]["mda_idx"].astype(int).tolist(),
                             s[top].astype(float).tolist()))
    return preds

# 운영용: 특정 광고 리스트만 빠르게 예측
def predict_for_specific_ads_fast(
    ranker, feat_cols, actx, ads_list, ad_id_list, K=10,
    topC=3, cat_topM=2, add_random=1,
    min_active_clk=300, min_active_conv=30
):
    # 숫자만, ads_list에 실제 존재하는 광고만 사용
    ad_id_list = pd.Series(pd.to_numeric(ad_id_list, errors="coerce")).dropna().astype(int)
    exist_ads = set(pd.to_numeric(ads_list["ads_idx"], errors="coerce").dropna().astype(int))
    ad_id_list = [a for a in ad_id_list if a in exist_ads]
    if not ad_id_list:
        return {}

    # 길이 맞춘 더미 TGT
    tgt_dummy = pd.DataFrame({"ads_idx": ad_id_list})
    tgt_dummy["mda_idx"] = pd.Series(dtype=int)

    return predict_with_per_ad_candidates_for_eval(
        ranker, feat_cols, actx, ads_list, tgt_dummy, K=K,
        topC=topC, cat_topM=cat_topM, add_random=add_random,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv
    )

# -------------------------------
# 8) GT 구성/지표
# -------------------------------
def make_gt_sets_and_gains(tgt: pd.DataFrame):
    g = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
           .agg(turn=("rpt_time_turn", lambda s: _to_num(s).sum())))
    gt_gain = {int(ad): {int(r["mda_idx"]): float(r["turn"]) for _, r in gg.iterrows()}
               for ad, gg in g.groupby("ads_idx")}
    gt_set  = {ad: set(d.keys()) for ad, d in gt_gain.items()}
    return gt_set, gt_gain

def _precision_at_k(gt_set, pred_list, k):
    if k <= 0: return np.nan
    k = min(k, len(pred_list))
    return sum(1 for a in pred_list[:k] if a in gt_set) / max(1, k)

def _recall_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    return sum(1 for a in pred_list[:k] if a in gt_set) / len(gt_set)

def _map_at_k(gt_set, pred_list, k):
    if len(gt_set) == 0: return np.nan
    hits, ap = 0, 0.0
    for i, a in enumerate(pred_list[:k], start=1):
        if a in gt_set:
            hits += 1; ap += hits / i
    return ap / min(len(gt_set), k)

def _ndcg_at_k_from_gains(gains_in_pred_order, k):
    k = min(k, len(gains_in_pred_order))
    dcg  = sum(g / math.log2(i+2) for i, g in enumerate(gains_in_pred_order[:k]))
    ideal_gains = sorted(gains_in_pred_order, reverse=True)[:k]
    idcg = sum(g / math.log2(i+2) for i, g in enumerate(ideal_gains))
    return dcg / idcg if idcg > 0 else 0.0

def _ndcg_at_k(gt_gain_map, pred_list, k):
    gains = [gt_gain_map.get(a, 0.0) for a in pred_list[:k]]
    return _ndcg_at_k_from_gains(gains, k)

def evaluate_with_gain(preds, gt_set, gt_gain, k=10):
    rows = []
    for ad, lst in preds.items():
        if ad not in gt_set: continue
        pred_mdas = [m for m,_ in lst]
        row = {
            "P": _precision_at_k(gt_set[ad], pred_mdas, k),
            "R": _recall_at_k(gt_set[ad], pred_mdas, k),
            "MAP": _map_at_k(gt_set[ad], pred_mdas, k),
            "NDCG": _ndcg_at_k(gt_gain.get(ad, {}), pred_mdas, k),
            "Hit": 1.0 if any(m in gt_set[ad] for m in pred_mdas[:k]) else 0.0
        }
        rows.append(row)
    if not rows:
        return {f"precision@{k}":0.0, f"recall@{k}":0.0,
                f"hit_rate@{k}":0.0, f"map@{k}":0.0, f"ndcg@{k}":0.0,
                "evaluated_ads":0}
    df = pd.DataFrame(rows)
    return {
        f"precision@{k}": float(df["P"].mean()),
        f"recall@{k}":    float(df["R"].mean()),
        f"hit_rate@{k}":  float(df["Hit"].mean()),
        f"map@{k}":       float(df["MAP"].mean()),
        f"ndcg@{k}":      float(df["NDCG"].mean()),
        "evaluated_ads":  int(len(df))
    }

def summarize_metrics(metrics_by_k: Dict[int, Dict[str, float]]) -> pd.DataFrame:
    rows = []
    for k, m in sorted(metrics_by_k.items()):
        rows.append([k, m.get(f"precision@{k}",0.0), m.get(f"recall@{k}",0.0),
                     m.get(f"hit_rate@{k}",0.0), m.get(f"map@{k}",0.0),
                     m.get(f"ndcg@{k}",0.0), m.get("evaluated_ads",0)])
    return pd.DataFrame(rows, columns=["K","Precision","Recall","HitRate","MAP","NDCG","#ads"])

# -------------------------------
# 9) 원클릭 파이프라인 (eval / serve)
# -------------------------------
PRESETS = {
    "eval":  dict(topC=5,  cat_topM=3, add_random=1, min_active_clk=300, min_active_conv=30,
                  max_per_ad=30, K_list=[5,10,20], K=20),
    "serve": dict(topC=3,  cat_topM=2, add_random=1, min_active_clk=300, min_active_conv=30,
                  max_per_ad=10, K_list=[10],       K=10),
}

def run_pipeline(ads_time: pd.DataFrame, ads_list: pd.DataFrame,
                 mode: str = "eval",
                 serve_ads: Optional[List[int]] = None,
                 tgt_days: int = 7,
                 max_train_ads: Optional[int] = 20000,
                 min_pos_in_tgt: int = 1):
    assert mode in ("eval", "serve")
    cfg = PRESETS[mode]

    # 1) 윈도우
    obs, tgt, start, end, cut = split_obs_tgt_month(ads_time, tgt_days=tgt_days)
    print(f"[Window] OBS:{start.date()}→{cut.date()} | TGT:{(cut+pd.Timedelta(days=1)).date()}→{end.date()}")

    # 2) 학습 데이터
    train_df, actx = prepare_training_attr_only(
        obs, tgt, ads_list,
        topC=cfg["topC"], add_random=cfg["add_random"], cat_topM=cfg["cat_topM"],
        min_active_clk=cfg["min_active_clk"], min_active_conv=cfg["min_active_conv"],
        max_train_ads=max_train_ads, min_pos_in_tgt=min_pos_in_tgt,
        max_per_ad=cfg["max_per_ad"]
    )
    print("train_df size:", len(train_df))

    # 3) 학습
    train_df = downsample_negatives_per_ad(train_df, ratio=4)
    ranker, feat_cols = train_attr_ranker(train_df)
    print("feat_cols used:", feat_cols)

    if mode == "eval":
        # 4) 평가
        preds = predict_with_per_ad_candidates_for_eval(
            ranker, feat_cols, actx, ads_list, tgt, K=cfg["K"],
            topC=cfg["topC"], cat_topM=cfg["cat_topM"], add_random=cfg["add_random"],
            min_active_clk=cfg["min_active_clk"], min_active_conv=cfg["min_active_conv"]
        )
        gt_set, gt_gain = make_gt_sets_and_gains(tgt)
        metrics = {k: evaluate_with_gain(preds, gt_set, gt_gain, k=k) for k in cfg["K_list"]}
        print(summarize_metrics(metrics))
        return dict(ranker=ranker, feat_cols=feat_cols, actx=actx, metrics=metrics, preds=preds)

    else:  # serve
        if not serve_ads:
            raise ValueError("mode='serve'일 때는 serve_ads 리스트가 필요합니다.")
        preds = predict_for_specific_ads_fast(
            ranker, feat_cols, actx, ads_list, serve_ads, K=cfg["K"],
            topC=cfg["topC"], cat_topM=cfg["cat_topM"], add_random=cfg["add_random"],
            min_active_clk=cfg["min_active_clk"], min_active_conv=cfg["min_active_conv"]
        )
        return dict(ranker=ranker, feat_cols=feat_cols, actx=actx, preds=preds)

# =======================
# 사용 예시
# =======================
# 1) 평가 모드
# out_eval = run_pipeline(ads_time, ads_list, mode="eval")
# print(out_eval["metrics"])

# 2) 운영 모드 (특정 광고만)
# out_serve = run_pipeline(ads_time, ads_list, mode="serve", serve_ads=[157236, 12345, 67890])
# print(out_serve["preds"].get(157236, [])[:10])


In [ ]:
out_serve = run_pipeline(ads_time, ads_list, mode="serve", serve_ads=[157236, 12345, 67890])
print(out_serve["preds"].get(157236, [])[:10])

[Window] OBS:2025-08-01→2025-08-24 | TGT:2025-08-25→2025-08-31
[Candidates] pairs=3,788 ads=637 active_mda=180
train_df size: 3788
feat_cols used: ['mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'price_reward_log', 'price_ratio']
[(563, 5.809610164607015), (634, -1.2333603951435752), (342, -5.135505007466736), (564, -5.667116268409439), (58, -5.785656754065143), (672, -6.107866299578369)]


# 분류모델

In [ ]:
import pandas as pd

def build_simple_features(ads_time, ads_list, ads_domain):
    # -------------------------
    # 1) 매체사별 클릭수, 전환수, 전환율
    # -------------------------
    mda_perf = (
        ads_time.groupby("mda_idx", as_index=False)
        .agg(
            mda_clicks=("rpt_time_clk", "sum"),
            mda_convs=("rpt_time_turn", "sum")
        )
    )
    mda_perf["mda_cvr"] = mda_perf["mda_convs"] / mda_perf["mda_clicks"].replace(0, pd.NA)

    # -------------------------
    # 2) 광고 속성 (카테고리, OS, 단가, 리워드 단가)
    # -------------------------
    ad_meta = ads_list[["ads_idx", "ads_category", "ads_os_type", 
                        "ads_contract_price", "ads_reward_price"]].copy()

    # -------------------------
    # 3) 광고별 도메인
    # -------------------------
    ad_domain = ads_domain[["ads_idx", "domain"]].copy()

    # -------------------------
    # 4) 합치기
    # -------------------------
    features = (
        ad_meta
        .merge(ad_domain, on="ads_idx", how="left")   # 광고에 도메인 붙이기
        .assign(key=1)                                # 전체 매체사와 조인하기 위한 키
        .merge(mda_perf.assign(key=1), on="key", how="left")
        .drop(columns="key")
    )

    return features

# === 실행 ===
X_simple = build_simple_features(ads_time, ads_list, ads_domain)


NameError: name 'ads_time' is not defined

In [44]:
def build_simple_features(ads_time, ads_list, ads_domain):

    # 1) 매체사별 클릭수, 전환수, 전환율
    mda_perf = (
        ads_time.groupby("mda_idx", as_index=False)
        .agg(
            mda_clicks=("rpt_time_clk", "sum"),
            mda_convs=("rpt_time_turn", "sum")
        )
    )
    mda_perf["mda_cvr"] = mda_perf["mda_convs"] / mda_perf["mda_clicks"].replace(0, pd.NA)

    # 2) 광고 속성 (카테고리, OS, 단가, 리워드 단가)
    ad_meta = ads_list[["ads_idx", "ads_category", "ads_os_type", 
                        "ads_contract_price", "ads_reward_price"]].copy()

    # 3) 광고별 도메인
    ad_domain = ads_domain[["ads_idx", "domain"]].copy()

    # 4) 합치기
    features = (
        ad_meta
        .merge(ad_domain, on="ads_idx", how="left")   # 광고에 도메인 붙이기
        .assign(key=1)                                # 전체 매체사와 조인하기 위한 키
        .merge(mda_perf.assign(key=1), on="key", how="left")
        .drop(columns="key")
    )

    return features

# === 실행 ===
X_simple = build_simple_features(ads_time, ads_list, ads_domain)


In [45]:
X_simple

,ads_idx,ads_category,ads_os_type,ads_contract_price,ads_reward_price,domain,mda_idx,mda_clicks,mda_convs,mda_cvr
0,160,1,2,170,150,NaN,12,224440,80663,0.359397
1,160,1,2,170,150,NaN,14,84025,22860,0.272062
2,160,1,2,170,150,NaN,18,5776,267,0.046226
3,160,1,2,170,150,NaN,22,80177,32994,0.411515
4,160,1,2,170,150,NaN,26,670,70,0.104478
...,...,...,...,...,...,...,...,...,...,...
84112366,446918,8,7,300,225,미디어/컨텐츠,1050,1,1,1.000000
84112367,446918,8,7,300,225,미디어/컨텐츠,1051,1,0,0.000000
84112368,446918,8,7,300,225,미디어/컨텐츠,1055,3,0,0.000000
84112369,446918,8,7,300,225,미디어/컨텐츠,1059,6,0,0.000000
